In [1]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ['https://www.googleapis.com/auth/generative-language.tuning']

def load_creds():
    """Converts `oauth-client-id.json` to a credential object.
    
    This function caches the generated tokens to minimize the use of the
    consent screen.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'ft_model_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

# load_creds()

In [2]:
import json
import google.generativeai as palm

my_config = json.load(open('config.json'))

creds = load_creds()
palm.configure(credentials=creds)

c:\Users\Pratham\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [79]:
text_model = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods][0].name
# argument_mapping_model = [m for m in palm.list_tuned_models()][0].name
argument_mapping_model = "tunedModels/argumentmappingmodel1-h21ndqfw3nfw"
embeddings_model = [m for m in palm.list_models() if 'embedText' in m.supported_generation_methods][0].name

In [28]:
arguments_description = json.load(open('./etc/refined_arguments_description.json'))

In [29]:
tools = json.load(open('./etc/tools.json'))
# models = palm.list_models()
# example_prompts = json.load(open('example_prompts.json'))["examples"]

In [81]:
arg_prompt = """
I have a argument for an function call. I want to summarize it and get its data type and possible values.
For Example:

Input:{"name": "ticket.severity","description": "Filters for tickets with any of the provided severities. Allowed values: blocker, high, low, medium","type": "array of strings"}
Ouput: {"desc": "Filters for tickets with any of the provided severities", "type": "Array[String]", "allowed": ["blocker", "high", "low", "medium"]}

Input:{"name": "query","description": "The search string, could be for example customer's name, part name, user name.","type": "string"}
Output: {"desc": "The search string, could be for example customer's name, part name, user name.", "type": "String"}

Input: {"name": "limit","description": "The maximum number of works to return. The default is '50'","type": "integer (int32)"}
Output: {"desc": "The maximum number of works to return.", "type": "Number"}

Input: {"name": "variance", "description": "The VAriance of the quantitiy. The default is '1'","type": "float (float32)"}
Output: {"desc": "The maximum number of works to return.", "type": "Number"}

Input: {"name": "applies_to_part", "description": "Filters for work belonging to any of the provided parts", "type": "array of strings", "example": [ "FEAT-123", "ENH-123", "PROD-123", "CAPL-123" ]}
Output: {"desc": "Filters for work belonging to any of the provided parts.", "type": "Array[String]", "example": ["FEAT-123", "ENH-123", "PROD-123", "CAPL-123"]}

solve for the following argument ensuring output is valid json:
Input: %s
Output:
"""

arguments_description = {}

def generate_argument_descriptions(tools, look_in = 'refined_arguments_description.json'):
    if os.path.exists('refined_arguments_description.json'):
        arguments_description = json.load(open(look_in, 'r'))
    else:
        arguments_description = {}
        for tool in tools["tools"]:
            arguments_description[tool["name"]] = []
            for argument in tool.get("arguments" ,[]):
                output = palm.generate_content(
                    model=text_model,
                    prompt=arg_prompt % argument,
                    temperature=0,
                    max_output_tokens=800,
                )
                arguments_description[tool["name"]].append({argument["name"]: eval(output.result)}) 
        json.dump(arguments_description, open(look_in, 'w'))
    return arguments_description

arguments_description = generate_argument_descriptions(tools)

In [30]:
def get_tools_description(tools, argument_descriptions):
    tools_description = ""
    for tool in tools["tools"]:
        tools_description += "\n" + f"{tool['name']}:{tool['description'].split('.')[0]}" 
        for argument in argument_descriptions[tool["name"]]:
            tools_description += " with args:"
            for arg, props in argument.items():
                tools_description += f"\n\t{arg}:{props['desc'].split('.')[0]}"
                # if 'example' in props.keys():
                #     tools_description += f" like: {props['example'][:2]}"
    return tools_description

print(get_tools_description(tools, arguments_description))


works_list:Returns a list of work items matching the request with args:
	applies_to_part:Filters for work belonging to any of the provided parts with args:
	created_by:Filters for work created by any of these users with args:
	owned_by:Filters for work owned by any of these users with args:
	issue.rev_orgs:Filters for issues with any of the provided Rev organizations with args:
	issue.priority:Filters for issues with any of the provided priorities with args:
	limit:The maximum number of works to return with args:
	stage.name:Filters for records in the provided stage(s) by name with args:
	ticket.needs_response:Filters for tickets that need a response with args:
	ticket.severity:Filters for tickets with any of the provided severities with args:
	ticket.source_channel:Filters for tickets with any of the provided source channels with args:
	types:Filters for work of the provided types
summarize_objects:Summarizes a list of objects with args:
	objects:List of objects to summarize
prioriti

In [15]:
segmentation_prompt = """
You have sentence which contains a task statement. You need to perform coreference resolution and split it into multiple sentences containing simpler atomic subtasks without fabricating any information. E.g.:
Input: Retrieve all tasks resembling TKT-123, condense the information, generate problem reports based on the summary, and arrange them in order of priority
Output: ["Get tasks resembling TKT-123","Summarize the tasks resembling TKT-123","Generate reports from summarized tasks resembling TKT-123","Prioritize the tasks resembling TKT-123"]

Input: Post an AI generated image on twitter. Monitor the number of likes and retweets. Perform analysis on the collected metrics.
Output: ["Post AI generated image on twitter","Monitor the AI generated post on twitter","Analyze monitored metrics of the AI generated post on twitter"]

Input: Generate product reccommendations for the user User-123 based on the summary of all posts made by them.
Output: ["Get all posts by User-123","Summarize the posts by User-123","Generate product reccommendations from summarized posts made by User-123"]  

Input: Add all issues P0 issues by User-123 to my sprint. Also create their summary.
Output: ["Get all P0 issues by User-123","Add P0 issues by User-123 to sprint","Summarize the P0 issues by User-123"]

Input: Could you get all my high priority work items?
Output: ["Get all my work items assigned high priority"]

Input: Given a transcript T, generate a summary S and perform sentiment analysis on S.
Output: ["Generate summary S from transcript T","Perform sentiment analysis on S"]

Input: Obtain details on all feature requests, generate a summary, and prioritize feature development.
Output: ["Get details on feature requests","Summarize the details on feature requests","Prioritize feature development from summarized details on feature requests"]

Input: Retrieve all tasks assigned to Team-A, condense the information, generate performance reports, and prioritize tasks.
Output: ["Get tasks assigned to Team-A","Summarize tasks assigned to Team-A","Generate performance reports from tasks assigned to Team-A","Prioritize tasks assigned to Team-A"]

Input: %s
Output:
"""

In [16]:
print(json.dumps(segmentation_prompt))

"\nYou have sentence which contains a task statement. You need to perform coreference resolution and split it into multiple sentences containing simpler atomic subtasks without fabricating any information. E.g.:\nInput: Retrieve all tasks resembling TKT-123, condense the information, generate problem reports based on the summary, and arrange them in order of priority\nOutput: [\"Get tasks resembling TKT-123\",\"Summarize the tasks resembling TKT-123\",\"Generate reports from summarized tasks resembling TKT-123\",\"Prioritize the tasks resembling TKT-123\"]\n\nInput: Post an AI generated image on twitter. Monitor the number of likes and retweets. Perform analysis on the collected metrics.\nOutput: [\"Post AI generated image on twitter\",\"Monitor the AI generated post on twitter\",\"Analyze monitored metrics of the AI generated post on twitter\"]\n\nInput: Generate product reccommendations for the user User-123 based on the summary of all posts made by them.\nOutput: [\"Get all posts by

In [84]:
response = palm.generate_content(
    model=text_model,
    prompt=segmentation_prompt % "Summarize high severity tickets from the customer UltimateCustomer"
)

In [85]:
response.result

'["Get high severity tickets from UltimateCustomer","Summarize the high severity tickets from UltimateCustomer"]'

In [86]:
for example in example_prompts:
    response = palm.generate_content(
        model=text_model,
        prompt=segmentation_prompt % example,
        temperature=0,
        max_output_tokens=800,
    )
    print(f"Input: {example}\nOutput: {response.result}\n")

Input: Summarize work items similar to don:core:dvrv-us-1:devo/0:issue/1
Output: ["Get work items similar to don:core:dvrv-us-1:devo/0:issue/1","Summarize the work items"]

Input: What is the meaning of life?
Output: ["Get the meaning of life"]

Input: Prioritize my P0 issues and add them to the current sprint
Output: ["Prioritize my P0 issues","Add my P0 issues to the current sprint"]

Input: Summarize high severity tickets from the customer UltimateCustomer
Output: ["Get high severity tickets from the customer UltimateCustomer","Summarize the high severity tickets from the customer UltimateCustomer"]

Input: What are my all issues in the triage stage under part FEAT-123? Summarize them.
Output: ["Get all my issues in the triage stage under part FEAT-123","Summarize the issues in the triage stage under part FEAT-123"]

Input: List all high severity tickets coming in from slack from customer Cust123 and generate a summary of them.
Output: ["Get all high severity tickets coming in from 

In [35]:
tool_getter_prompt = """
Given a task statement, you need to identify the tool that can be used to perform the task. If no such tool is present you return "None". You are to return only a single tool name that is most suitable complete the given task (the final task), keeping in context the tool name and the tool arguments. Examples:
Tools:
tool_A:create action items from text
\ttext:The text from which action items are to be created
tool_B:Add action items to sprint
\titems:The action items to be added to the sprint
Input: Add action items from customer meeting transcript T to current sprint
Output: tool_B

Tools:
tool_W:get work item details
\twork_id:The id for which details are to be fetched
tool_X:Given a search string, returns the id of a matching object in the system of record with args:
\tquery:The search string, could be for example customer’s name, part name, user name
Input: Get objects related to customer UltimateCustomer
Output: tool_X

Now solve for the given case:
Tools: 
%s
Task: %s
Ouput:
"""
print(json.dumps(tool_getter_prompt))


"\nGiven a task statement, you need to identify the tool that can be used to perform the task. If no such tool is present you return \"None\". You are to return only a single tool name that is most suitable complete the given task (the final task), keeping in context the tool name and the tool arguments. Examples:\nTools:\ntool_A:create action items from text\n\ttext:The text from which action items are to be created\ntool_B:Add action items to sprint\n\titems:The action items to be added to the sprint\nInput: Add action items from customer meeting transcript T to current sprint\nOutput: tool_B\n\nTools:\ntool_W:get work item details\n\twork_id:The id for which details are to be fetched\ntool_X:Given a search string, returns the id of a matching object in the system of record with args:\n\tquery:The search string, could be for example customer\u2019s name, part name, user name\nInput: Get objects related to customer UltimateCustomer\nOutput: tool_X\n\nNow solve for the given case:\nToo

In [88]:
tools_description = get_tools_description(tools, arguments_description)

example_tasks = ["Get the meaning of life"]
example_tasks = eval(response.result)
example_tasks = [
    "Get all high severity tickets coming in from slack from customer Cust123",
    "Summarize the high severity tickets coming in from slack from customer Cust123",
]
example_tasks = ["Summarize my P0 issues", "Add my P0 issues to the current sprint"]
# example_tasks =  ["Get all my issues in the triage stage under part FEAT-123","Summarize the issues in the triage stage under part FEAT-123"]
# example_tasks =  ["Get work items similar to don:core:dvrv-us-1:devo/0:issue/1","Summarize the work items"]


def get_tools_for_tasks(tasks):
    output = []
    for task in tasks:
        response = palm.generate_content(
            model=text_model,
            prompt=tool_getter_prompt % (tools_description, task),
            temperature=0,
            max_output_tokens=800,
        )
        result = response.result
        if result == "None":
            return []
        output.append((task, response.result))
    return output

# print(tool_getter_prompt % (tools_description, example_tasks[0]))
task_and_tool = get_tools_for_tasks(example_tasks)

In [89]:
print(task_and_tool)

[('Summarize my P0 issues', 'summarize_objects'), ('Add my P0 issues to the current sprint', 'add_work_items_to_sprint')]


#### Fine tune on more examples like the ones below to improve accuracy

In [9]:
subtask_solution_prompt = """
You're a issue management bot. Given a tool and directive, you strictly use values in the Past Actions or the directive to map to argument values mentioned in the "Tool" Section. If there is some information missing you return the action to get the missing information. Eg:

Directive:Summarize the work items similar to TKT-123

Tool:summarize_objects with args:
- objects (Array[Object]): The objects to summarize

Past Actions:
- Prioritize the work items similar to TKT-123:prioritize_work_items
- Get all similar work items to TKT-123: get_similar_work_items
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint

Output:{"result":[("objects","get_similar_work_items")]}
---
Directive:Get my P0 issues on part DEV-PART-123

Tool:get_items with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- owned_by (Array[String]): Filters for work owned by any of these users
- created_by (Array[String]): Filters for work created by any of these users
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ['p0', 'p1', 'p2', 'p3']

Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint

Output:{"result":[("owned_by","who_am_i"),("issue.priority","p0"),("applies_to_part","DEV-PART-123")]}
---
Directive: Close all unresolved issues

Tool: close_issues with args:
- issues (Array[Object]): List of issues to close

Past Actions:

Output: {"missing_action": "Get the list of unresolved issues"}
---
Directive: Move all tasks to the next sprint

Tool: move_issues with args:
- issues (Array[Object]): List of issues to move
- destination_sprint (String): The sprint to move the issues to

Past Actions:
- Get the ID of the current User: who_am_i
- Get the next sprint: get_next_sprint

Output: {"missing_action": "Get the list of all tasks"}
---
Directive: Prioritize my P0 issues

Tool:prioritize_objects with args:
- objects (Array[Object]): The objects to prioritze

Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint

Output: 
"""
print(subtask_solution_prompt)


You're a issue management bot. Given a tool and directive, you strictly use values in the Past Actions or the directive to map to argument values mentioned in the "Tool" Section. If there is some information missing you return the action to get the missing information. Eg:

Directive:Summarize the work items similar to TKT-123

Tool:summarize_objects with args:
- objects (Array[Object]): The objects to summarize

Past Actions:
- Prioritize the work items similar to TKT-123:prioritize_work_items
- Get all similar work items to TKT-123: get_similar_work_items
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint

Output:{"result":[("objects","get_similar_work_items")]}
---
Directive:Get my P0 issues on part DEV-PART-123

Tool:get_items with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- owned_by (Array[String]): Filters for work owned by any of these users
- created_by (Array[String]): Filters for wor

In [91]:
class KnowledgeItem:
    description: str
    tool: str

    def __init__(self, description: str, tool: str, arg_mapping: tuple = None) -> None:
        self.description = description
        self.tool = tool
        if arg_mapping:
            self.arg_mapping = arg_mapping
        else:
            self.arg_mapping = ()

    def summarize(self) -> str:
        return self.description + ": " + self.tool

    def __str__(self) -> str:
        return f"Know <{self.description} from [{self.tool}]>"

    def __hash__(self) -> int:
        return hash(str(self))

    def __repr__(self) -> str:
        return str(self)
    
    

def get_base_knowledge(tools, arguments_description):
    knowledge = []

    l = list(arguments_description.keys())

    for tool in arguments_description:
        if len(arguments_description[tool]) == 0:
            tool_names = [t["name"] for t in tools["tools"]]
            index = tool_names.index(tool)
            tool_description = tools["tools"][index]["description"]
            knowledge.append(KnowledgeItem(tool_description, tool))

    return knowledge


base_knowledge = get_base_knowledge(tools, arguments_description)

In [105]:
response = palm.generate_content(
    model=text_model,
    prompt=subtask_solution_prompt,
    temperature=0,
    max_output_tokens=800,
)

In [92]:
def elaborate_args(args: list[dict]):
    response = ""
    primary_count = 0
    for arg in args:
        for name, props in arg.items():
            response += "\n- "
            response += f"{name} ({props['type']}): {props['desc']}"
            primary_count += 1
            if 'allowed' in props.keys():
                response += f" allowing: {props['allowed']}"

    return response


def get_instruction_prompt(instruction, arguments_description, knowledge):
    directive = instruction[0]
    tool_to_be_used = instruction[1]

    tool_arguments = arguments_description[tool_to_be_used]

    prompt = "Solve the 'Directive' with the given 'Tool'. Use values in 'Past Actions' or the provided directive and map values to arguments in the Tool. In case of missing info return a directive to get the missing info required to get missing info."
    prompt += f"\nDirective:{directive}\nTool: {tool_to_be_used} with args:{elaborate_args(tool_arguments)}"
    
    prompt += "\nPast Actions:"
    for knowledge_item in knowledge[::-1]:
        prompt += f"\n- {knowledge_item.summarize()}"

    return prompt

In [93]:
print(get_instruction_prompt(task_and_tool[0], arguments_description, base_knowledge))

Solve the 'Directive' with the given 'Tool'. Use values in 'Past Actions' or the provided directive and map values to arguments in the Tool. In case of missing info return a directive to get the missing info required to get missing info.
Directive:Summarize my P0 issues
Tool: summarize_objects with args:
- objects (Array[Object]): List of objects to summarize
Past Actions:
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id


In [10]:
d = {}
# d["argument_processing_prompt"] = arg_prompt
# d["segmentation_prompt"] = segmentation_prompt
d["tool_getter_prompt"] = tool_getter_prompt
# d["subtask_solution_prompt"] = subtask_solution_prompt

json.dump(d, open('prompts.json', 'w'))

In [96]:
from collections import deque

instructions = deque(task_and_tool)


def complete_task(instructions: deque, tools, arguments_description):
    knowledge = get_base_knowledge(tools, arguments_description)

    total_input_len = 0
    total_output_len = 0
    steps = 0

    while len(instructions) > 0:
        if steps > 6:
            break
        steps += 1
        instruction = instructions[0]
        response = palm.generate_content(
            model=argument_mapping_model,
            prompt=get_instruction_prompt(instruction, arguments_description, knowledge),
            temperature=0,
        )
        print(f"Input: {instruction}\nOutput: {response.result}")
        response = eval(response.result)

        # tokens = palm.count_text_tokens(
        #     model=argument_mapping_model,
        #     prompt=get_instruction_prompt(instruction, arguments_description, knowledge),
        # )
        # total_len += tokens['token_count']

        total_input_len += len(get_instruction_prompt(instruction, arguments_description, knowledge))
        total_output_len += len(response.result)

        if len(response.get("missing_action", "")) > 0:
            print(f"Missing action: {response['missing_action']}")

            tool_for_missing_action = get_tools_for_tasks([response["missing_action"]])

            print(f"Tool for missing action: {tool_for_missing_action}")

            if len(tool_for_missing_action) > 0:
                instructions.appendleft((response["missing_action"], tool_for_missing_action[0][1]))
        else:
            print(f"Result: {response.get('result', [])}")
            instructions.popleft()
            knowledge.append(KnowledgeItem(instruction[0], instruction[1], response["result"]))
        print()
    
    return knowledge


# print(instructions)

results = complete_task(instructions, tools, arguments_description)

Input: ('Summarize my P0 issues', 'summarize_objects')
Output: {'missing_action': 'Get my P0 issues'}
Missing action: Get my P0 issues
Tool for missing action: [('Get my P0 issues', 'works_list')]

Input: ('Get my P0 issues', 'works_list')
Output: {'result': [('owned_by', ['who_am_i']), ('issue.priority', ['p0'])]}
Result: [('owned_by', ['who_am_i']), ('issue.priority', ['p0'])]

Input: ('Summarize my P0 issues', 'summarize_objects')
Output: {'result': [('objects', 'works_list')]}
Result: [('objects', 'works_list')]

Input: ('Add my P0 issues to the current sprint', 'add_work_items_to_sprint')
Output: {'result': [('work_ids', 'works_list'), ('sprint_id', 'get_sprint_id')]}
Result: [('work_ids', 'works_list'), ('sprint_id', 'get_sprint_id')]

Total tokens: 834


In [97]:
from copy import deepcopy

def topo_sort(knowledge: list[KnowledgeItem], tools, arguments_description) -> list:
    """Returns a topologically sorted list of knowledge items."""
    knowledge = deepcopy(knowledge)
    final_goal = knowledge[-1]
    for item in knowledge:
        neighbors = set()
        for arg in item.arg_mapping:
            # print(arg)
            if (
                isinstance(arg[1], list)
                or isinstance(arg[1], tuple)
                or isinstance(arg[1], set)
                or isinstance(arg[1], dict)
            ):
                nbs = [k_item for k_item in knowledge if k_item.tool in arg[1]]
                neighbors.update(nbs)
            else:
                nbs = [k_item for k_item in knowledge if k_item.tool == arg[1]]
                neighbors.update(nbs)
        item.neighbors = neighbors

    def topo_sort_util(k_item: KnowledgeItem, visited: set, stack: list):
        visited.add(k_item)
        for neighbor in k_item.neighbors:
            if neighbor not in visited:
                topo_sort_util(neighbor, visited, stack)
        stack.append(k_item)

    visited = set()
    stack = []

    topo_sort_util(final_goal, visited, stack)

    # stack = stack[::-1]
    solution = []
    for item in stack:
        tool_ordering = [k_item.tool for k_item in stack]
        tool_args = [list(d.keys())[0] for d in  arguments_description[item.tool]]
        solution_item = {}
        solution_item["tool_name"] = item.tool
        solution_item["arguments"] = []
        for arg in item.arg_mapping:
            if arg[0] not in tool_args:
                continue
            value = arg[1]
            if isinstance(arg[1], list):
                for i in range(len(arg[1])):
                    if arg[1][i] in tool_ordering:
                        value[i] = f"$$PREV[{tool_ordering.index(arg[1][i])}]"
            elif arg[1] in tool_ordering:
                value = f"$$PREV[{tool_ordering.index(arg[1])}]"

            solution_item["arguments"].append(
                {"argument_name": arg[0], "argument_value": value}
            )
        solution.append(solution_item)

    return solution


sol = topo_sort(results, tools, arguments_description)
print(json.dumps(sol, indent=2))

[
  {
    "tool_name": "who_am_i",
    "arguments": []
  },
  {
    "tool_name": "works_list",
    "arguments": [
      {
        "argument_name": "owned_by",
        "argument_value": [
          "$$PREV[0]"
        ]
      },
      {
        "argument_name": "issue.priority",
        "argument_value": [
          "p0"
        ]
      }
    ]
  },
  {
    "tool_name": "get_sprint_id",
    "arguments": []
  },
  {
    "tool_name": "add_work_items_to_sprint",
    "arguments": [
      {
        "argument_name": "work_ids",
        "argument_value": "$$PREV[1]"
      },
      {
        "argument_name": "sprint_id",
        "argument_value": "$$PREV[2]"
      }
    ]
  }
]


In [98]:
my_base_examples = [
"""
Directive:Summarize the high severity tickets coming in from slack from customer Cust123
Tool: summarize_objects with args:
- objects (Array[Object]): List of objects to summarize
Past Actions:
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
Output: {"result": [], "missing_action": "Get high severity tickets coming in from slack from customer Cust123"}
""",
"""
Directive:Summarize the high severity tickets coming in from slack from customer Cust123
Tool: summarize_objects with args:
- objects (Array[Object]): List of objects to summarize
Past Actions:
- Get the high severity tickets coming in from slack from customer Cust123: get_high_severity_tickets: works_list
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
Output: {"result": [("objects", "works_list")], "missing_action": ""}
""",
"""
Directive: Summarize the work items similar to TKT-123
Tool:summarize_objects with args:
- objects (Array[Object]):The objects to summarize
Past Actions:
- Prioritize the work items similar to TKT-123: prioritize_work_items
- Get all similar work items to TKT-123: get_similar_work_items
- the ID of the current User: who_am_i
- the current sprint:get_current_sprint
Output:{"result":[("objects","get_similar_work_items")],"missing_action":""}
""",
"""
Directive: Get my P0 issues on part DEV-PART-123
Tool:get_items with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- owned_by (Array[String]): Filters for work owned by any of these users
- created_by (Array[String]): Filters for work created by any of these users 
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"] (optional)
Past Actions:
- the ID of the current User: who_am_i
Output:{"result": [("owned_by",["who_am_i"]),("issue.priority",["p0"]),("applies_to_part",["DEV-PART-123"])],"missing_action": ""}
""",
"""
Directive: Summarize my P0 issues
Tool:summarize_objects with args:
- objects (Array[Object]):The objects to summarize
Past Actions:
- the ID of the current User: who_am_i
Output: {"result": [],"missing_action": "Get my P0 issues"}
""",
"""
Directive: Prioritize my P0 issues
Tool:prioritize_objects with args:
- objects (Array[Object]): The objects to summarize
Past Actions:
- the ID of the current User: who_am_i
Output: {"result": [],"missing_action": "Get my P0 issues"}
""",
"""
Directive: Send an email to User-123 regarding issue I-123.
Tool: send_email with args:
- to (String): The email address of the recipient
- from (String): The email address of the sender
- subject (String): The subject of the email (optional)
- body (String): The body of the email
Past Actions:
- Reutrn user details of User-123: get_user_details
Output: {"result": [], "missing_action": "Summarize issue I-123"}
""",
"""
Directive: Get all issues
Tool:get_items with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- owned_by (Array[String]): Filters for work owned by any of these users
- created_by (Array[String]): Filters for work created by any of these users 
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"] (optional)
Past Actions:
Output:{"result": [],"missing_action": ""}
""",
"""
Directive: Send an email to User-123 regarding issue I-123.
Tool: send_email with args:
- to (String): The email address of the recipient
- subject (String): The subject of the email (optional)
- body (String): The body of the email
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result": [], "missing_action": "Get user details of User-123"}
""",
"""
Directive:Get my P0 issues on all objects containing the word "AI"
Tool:get_items with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- owned_by (Array[String]): Filters for work owned by any of these users
- created_by (Array[String]): Filters for work created by any of these users 
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"] (optional)
Past Actions:
- the ID of the current User: who_am_i
Output:{"result": [],"missing_action": "Get all parts containing the word 'AI'"}
""",
"""
Directive: Get my P0 issues on all objects containing the word "AI"
Tool:get_items with args:
- applies_to_part (String): Filters for work belonging to any of the provided parts.
- owned_by (String): Filters for work owned by any of these users
- created_by (String): Filters for work created by any of these users 
- issue.priority (String): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"] (optional)
Past Actions:
- Get all parts containing the word "AI": get_objects_by_keyword
- the ID of the current User: who_am_i
Output:{"result": [("owned_by","who_am_i"),("issue.priority","p0"),("applies_to_part","get_objects_by_keyword")],"missing_action": ""}
""",
"""
Directive: Send an email to User-123 regarding issue I-123.
Tool: send_email with args:
- to (String): The email address of the recipient
- from (String): The email address of the sender
- subject (String): The subject of the email (optional)
- body (String): The body of the email
Past Actions:
- Get user details of User-123: get_user_details
- Summarize issue I-123: summarize_objects
- Returns the ID of the current User: who_am_i
Output: {"result": [("to", "get_user_details"), ("from", "who_am_i"), ("subject", "Regarding issue I-123"), ("body", "summarize_objects")],"missing_action": ""}
""",
"""
Directive: Dial up the user User-123
Tool: call_user with args:
- contact_details (Object): The contact details of the user
Past Actions:
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [], "missing_action": "Get user details of User-123"}
""",
"""
Directive: Dial up the user User-123
Tool: call_user with args:
- contact_details (Object): The contact details of the user
Past Actions:
- Get user details of User-123: get_user_details
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [("contact_details", "get_user_details")], "missing_action": ""}
""",
"""
Directive: Pay user User-123 his balance amount
Tool: pay_user with args:
- contact_details (Object): The contact details of the user
- amount (Number): The amount to pay the user
Past Actions:
- Get balance amount of User-123: get_user_balance
- Get user details of User-123: get_user_details
- the current sprint: get_current_sprint
Output: {"result": [("contact_details", "get_user_details"), ("amount", "get_user_balance")], "missing_action": ""}
""",
"""
Directive: Pay user User-123 his balance amount
Tool: pay_user with args:
- contact_details (Object): The contact details of the user
- amount (Number): The amount to pay the user
Past Actions:
- Returns user details of User-123: get_user_details
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [], "missing_action": "Get balance amount of User-123"}
""",
"""
Directive: Pay user User-123 his $100
Tool: pay_user with args:
- contact_details (Object): The contact details of the user
- amount (Number): The amount to pay the user
Past Actions:
- Returns user details of User-123: get_user_details
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [("contact_details", "get_user_details"), ("amount", 100)], "missing_action": ""}
""",
"""
Directive: Close all unresolved issues
Tool: close_issues with args:
- issues (Array[Object]): List of issues to close
Past Actions:
Output: {"result": [], "missing_action": "Get the list of unresolved issues"}
""",
"""
Directive: Move all tasks to the next sprint
Tool: move_issues with args:
- issues (Array[Object]): List of issues to move
- destination_sprint (String): The sprint to move the issues to
Past Actions:
- the ID of the current User: who_am_i
- the next sprint: get_next_sprint
Output: {"result": [], "missing_action": "Get the list of all tasks"}
""",
"""
Directive: Move all unresolved issues to the next sprint
Tool: move_issues with args:
- issues (Array[Object]): List of issues to move
- destination_sprint (String): The sprint to move the issues to
Past Actions:
- Returns List of unresolved issues: get_unresolved_issues
- Get the ID of the current User: who_am_i
- Return the next sprint: get_next_sprint
Output: {"result": [("issues", "get_unresolved_issues"), ("destination_sprint", "get_next_sprint")], "missing_action": ""}
""",
"""
Directive: Get my issues on all objects made by MAN-123
Tool:get_items with args:
- applies_to_part (String): Filters for work belonging to any of the provided parts.
- owned_by (String): Filters for work owned by any of these users
- created_by (String): Filters for work created by any of these users 
- issue.priority (String): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"] (optional)
Past Actions:
- Returns all parts containing the word "AI": get_objects_by_keyword
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output:{"result": [("owned_by","who_am_i"),("created_by","MAN-123")],"missing_action": ""}
""",
"""
Directive: Get my issues on all objects made by my Manager
Tool:get_items with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- owned_by (Array[String]): Filters for work owned by any of these users
- created_by (Array[String]): Filters for work created by any of these users 
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"] (optional)
Past Actions:
- Get all parts containing the word "AI": get_objects_by_keyword
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output:{"result": [], "missing_action": "Get user details of my Manager"}
""",
"""
Directive: Get issues made by users in my team
Tool:get_items with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- owned_by (Array[String]): Filters for work owned by any of these users
- created_by (Array[String]): Filters for work created by any of these users 
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"] (optional)
Past Actions:
Output:{"result": [],"missing_action": "Get user details of my team"}
""",
"""
Directive: Get issues made by users U-123 and U-456
Tool:get_items with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- owned_by (Array[String]): Filters for work owned by any of these users
- created_by (Array[String]): Filters for work created by any of these users 
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"] (optional)
Past Actions:
- Get all parts containing the word "AI": get_objects_by_keyword
- the ID of the current User: who_am_i
Output:{"result": [("created_by", ["U-123", "U-456"])],"missing_action": ""}
""",
"""
Directive: Get issues owned by users U-123 and U-456 and with priorities P0 and P1
Tool:get_work_items with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- owned_by (Array[String]): Filters for work owned by any of these users
- created_by (Array[String]): Filters for work created by any of these users 
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"] (optional)
Past Actions:
- Get all parts containing the word "AI": get_objects_by_keyword
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output:{"result": [("created_by", ["U-123", "U-456"]), ("issue.priority", ["p0", "p1"])],"missing_action": ""}
""",
"""
Directive: Summarize my open tasks
Tool: summarize_objects with args:
- objects (Array[Object]): The objects to summarize
Past Actions:
Output: {"result": [], "missing_action": "Get my open tasks"}
""",
"""
Directive: Assign a QA tester to Project P-789
Tool: assign_user with args:
- task_key (String): The key of the task to assign
- user_key (Array[String]): The key of the QA tester to assign to the issue
Past Actions:
- List of available QA testers: get_available_qa_testers
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [("task_key", "P-789"), ("user_key", ["get_available_qa_testers"])], "missing_action": ""}
""",
"""
Directive: Create a new high-priority issue from action item creted from T in the current backlog
Tool:create_issue with args:
- summary (String): A brief, one-sentence description of the issue
- description (String): A detailed description of the issue
- priority (String): Priority of the issue allowing: ["High", "Medium", "Low", "Blocker"]
- backlog (String): The backlog to add the issue to (optional)
Past Actions:
- List of available priorities: get_available_priorities
- Get action items created from T: get_action_items_from_text
- Get Current backlog: get_backlog_issues
- Get the ID of the current User: who_am_i
Output: {"result": [("summary", ""), ("description", ""), ("priority", "High"), ("backlog", "get_backlog_issues")], "missing_action": ""}
""",
"""
Directive: Create an action items created from transcript T
Tool: create_actionable_tasks_from_text with args:
- text (String):The text from which the actionable insights need to be created
Past Actions:
- List of available priorities: get_available_priorities
- Get Current backlog: get_backlog_issues
- Get the ID of the current User: who_am_i
Output: {"result": [("text", "T")], "missing_action": ""}
""",
"""
Directive: Assign developers to all tasks in the backlog
Tool: assign_users with args:
- tasks (Array[Object]): List of tasks to assign
- users (Array[String]): List of developers to assign to the tasks
Past Actions:
- List of developers: get_available_developers
- List of tasks in the backlog: get_backlog_tasks
- the ID of the current User: who_am_i
Output: {"result": [("tasks", "get_backlog_tasks"), ("users", ["get_available_developers"])], "missing_action": ""}
""",
"""
Directive: Summarize the tasks completed by my team
Tool: summarize_objects with args:
- objects (Array[Object]): The objects to summarize
Past Actions:
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [], "missing_action": "Get completed tasks by my team"}
""",
"""
Directive: Create a new issue with the summary "Fix bug in AI" and the description "This is a bug in AI"
Tool: create_issue with args:
- summary (String): A brief, one-sentence description of the issue
- description (String): A detailed description of the issue
- priority (String): Priority of the issue allowing: ["High", "Medium", "Low", "Blocker"]
- backlog (String): The backlog to add the issue to (optional)
Past Actions:
- List of available priorities: get_available_priorities
- Get the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [("summary", "Fix bug in AI"), ("description", "This is a bug in AI")], "missing_action": ""}
""",
"""
Directive: Close all unresolved support tickets
Tool: close_issues with args:
- issues (Array[Object]): List of issues to close
Past Actions:
- Get the list of all users: get_all_users
Output: {"result": [], "missing_action": "Get the list of unresolved support tickets"}
""",
"""
Directive: Close all unresolved bugs in the system
Tool: close_issues with args:
- issues (Array[Object]): List of issues to close
- issue.type (String): Filters for issues by the type ["Bug", "Feature", "Optimisation"] (optional)
- issue.status (String): Filters for issues by the status allowing ["Unresolved", "Resolved", "Abandoned"] (optional)
Past Actions:
- Get the list of all bugs: get_all_bugs
Output: {"result": [("issues", ["get_all_bugs"]), ("issue.type", ["Bug"]), ("issue.status", ["Unresolved"])], "missing_action": ""}
""",
"""
Directive: Assign a UI/UX designer to tasks in sprint S-456
Tool: assign_user with args:
- task_keys (Array[String]): The keys of the tasks to assign
- user_key (Array[String]): The key of the UI/UX designer to assign to the issue
Past Actions:
- List of available UI/UX designers: get_available_workers
- Get tasks in sprint S-456: get_tasks_in_sprint
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [("task_keys", "get_tasks_in_sprint"), ("user_key", ["get_available_workers"])], "missing_action": ""}
""",
"""
Directive: Assign a UI/UX designer to tasks in sprint S-456
Tool: assign_user with args:
- task_keys (Array[String]): The keys of the tasks to assign
- user_key (Array[String]): The key of the UI/UX designer to assign to the issue
Past Actions:
- List of available UI/UX designers: get_available_workers
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [], "missing_action": "Get tasks in sprint S-456"}
""",
"""
Directive: Create a new Blocking issue with the summary "Implement feature X" and the description "Details about implementing feature X"
Tool:create_issue with args:
- summary (String): A brief, one-sentence description of the issue
- description (String): A detailed description of the issue
- priority (String): Priority of the issue allowing: ["High", "Medium", "Low", "Blocker"]
- backlog (String): The backlog to add the issue to (optional)
Past Actions:
- List of available priorities: get_available_priorities
- Get the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [("summary", "Implement feature X"), ("description", "Details about implementing feature X"), ("priority", "Blocker")], "missing_action": ""}
""",
"""
Directive: Commit my changes to the current repository on a branch named "feature-123"
Tool:commit_changes with args:
- repository (String): The repository to commit to
- description (String): A detailed description of the commit
- branch (String): The branch to commit to (optional) 
Past Actions:
- Get the changes summary: get_changes_summary
- Get the name of the current branch: get_current_branch
- Get the name current repository: get_current_repository
Output: {"result": [("repository", "get_current_repository"), ("branch", "feature-123"), ("description", "get_changes_summary")], "missing_action": ""}
""",
"""
Directive: Commit my changes to the current repository on a branch named "feature-123"
Tool:commit_changes with args:
- repository (String): The repository to commit to
- description (String): A detailed description of the commit
- branch (String): The branch to commit to (optional) 
Past Actions:
- Get the name of the current branch: get_current_branch
- Get the name current repository: get_current_repository
Output: {"result": [], "missing_action": "Get the commit change description summary"}
""",
"""
Directive: Move all files with names starting with "feature-file" to the folder "folder-123" 
Tool:move_files with args:
- files (Array[String]): The files to move
- destination (String): The destination folder
Past Actions:
- Get files with names starting with "feature-file": get_files_by_name
- Get the name of the current branch: get_current_branch
- Get the name current repository: get_current_repository
Output: {"result": [("files", ["get_files_by_name"]), ("destination", "folder-123")], "missing_action": ""}
""",
"""
Directive: Move all files with names starting with 'feature-file' to the folder 'folder-123' 
Tool:move_files with args:
- files (Array[String]): The files to move
- destination (String): The destination folder
- delete (Boolean): Whether to delete the files after moving them (optional)
- overwrite (Boolean): Whether to overwrite the files if they already exist at destination (optional)
Past Actions:
- Get the id of the current user: who_am_i
- Get the current sprint: get_current_sprint
- Get the name of the current branch: get_current_branch
- Get the name current repository: get_current_repository
Output: {"result": [], "missing_action": "Get all files with names starting with 'feature-file'"}
""",
"""
Directive: Get all files with names starting with "feature-file"
Tool:get_files_by_name with args:
- name (String): The name of the files to get
Past Actions:
- Get the id of the current user: who_am_i
- Get the current sprint: get_current_sprint
- Get the name of the current branch: get_current_branch
- Get the name current repository: get_current_repository
Output: {"result": [("name", "feature-file")], "missing_action": ""}
""",
"""
Directive: Get the current sprint
Tool:get_current_sprint with args:
Past Actions:
- Get the id of the current user: who_am_i
- Get the id of the current sprint: get_current_sprint
- Returns the name of the current branch: get_current_branch
- Returns the name current repository: get_current_repository
Output: {"result": [], "missing_action": ""}
""",
"""
Directive: Summarize all the completed tasks from the last sprint
Tool: summarize_objects with args:
- objects (Array[Object]): The objects to summarize
Past Actions:
- Get the id of the current user: who_am_i
- Get the id of the current sprint: get_current_sprint
- Get the name of the current branch: get_current_branch
Output: {"result": [], "missing_action": "Get all the completed tasks from the last sprint"}
""",
"""
Directive: Summarize all the completed tasks from the last sprint
Tool: summarize_objects with args:
- objects (Array[Object]): The objects to summarize
Past Actions:
- Get the id of the current user: who_am_i
- Get the id of the current sprint: get_current_sprint
- Returns all completed tasks from the last sprint: get_completed_tasks
- Get last sprint: get_last_sprint
- Get the name of the current branch: get_current_branch
Output: {"result": [("objects", ["get_completed_tasks"])], "missing_action": ""}
""",
"""
Directive:Summarize my P0 issues
Tool: summarize_objects with args:
- objects (Array[Object]): List of objects to summarize
Past Actions:
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
Output: {"result": [], "missing_action": "Get my P0 issues"}
""",
"""
Directive:Get all high severity tickets coming in from slack from customer Cust123
Tool: works_list with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- created_by (Array[String]): Filters for work created by any of these users
- owned_by (Array[String]): Filters for work owned by any of these users
- issue.rev_orgs: Filters for issues with any of the provided Rev organizations (optional)
- issue.priority: Filters for issues with any of the provided priorities (optional) allowing: ['p0', 'p1', 'p2', 'p3']
- limit: The maximum number of works to return. (optional)
- stage.name: Filters for records in the provided stage(s) by name (optional)
- ticket.needs_response: Filters for tickets that need a response (optional)
- ticket.severity: Filters for tickets with any of the provided severities (optional) allowing: ['blocker', 'high', 'low', 'medium']
- ticket.source_channel: Filters for tickets with any of the provided source channels (optional)
- types: Filters for work of the provided types. (optional) allowing: ['issue', 'ticket', 'task']
Past Actions:
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
Output:{'result': [], 'missing_action': 'Get all objects related to customer Cust123'}
"""
]

In [99]:
print(len(my_base_examples))

47


In [100]:
import csv
import random


def convert_to_csv(fine_tune_examples, file_name):
    count_empty = 0
    csv_file_name = file_name

    data = []
    random.shuffle(fine_tune_examples)

    for example in fine_tune_examples:
        input = "Solve the 'Directive' with the given 'Tool'. Use values in 'Past Actions' or the provided directive and map values to arguments in the Tool. In case of missing info return a directive to get the missing info required to get missing info.\n"
        input = input + example.split("Output:")[0].strip()
        output = example.split("Output:")[1].strip()
        try:
            output = eval(output)
            if len(output.get("missing_action", "")) == 0:
                output.pop("missing_action", None)
                count_empty += 1
            elif len(output.get("result", [])) == 0:
                output.pop("result", None)
            data.append((input, str(output)))
        except Exception as e:
            print(e)
            print(type(output), output)
            # print()
            continue


    # Writing to CSV file
    with open(csv_file_name, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        
        # Write data from tuples
        for directive, tool in data:
            csv_writer.writerow([directive, tool])

    print(f'CSV file "{csv_file_name}" has been created.')
    print(f"Examples with no missing action: {count_empty}")

convert_to_csv(my_base_examples, "my_base_examples.csv")

CSV file "my_base_examples.csv" has been created.
Examples with no missing action: 26


In [117]:
my_base_examples_final = [
"""
Directive:Get my P0 issues
Tool: works_list with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- created_by (Array[String]): Filters for work created by any of these users.
- owned_by (Array[String]): Filters for work owned by any of these users.
- issue.rev_orgs (Array[String]): Filters for issues with any of the provided Rev organizations
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ['p0', 'p1', 'p2', 'p3']
- limit (Number): The maximum number of works to return.
- stage.name (Array[String]): Filters for records in the provided stage(s) by name
- ticket.needs_response (Boolean): Filters for tickets that need a response
- ticket.severity (Array[String]): Filters for tickets with any of the provided severities allowing: ['blocker', 'high', 'low', 'medium']
- ticket.source_channel (Array[String]): Filters for tickets with any of the provided source channels allowing: []
- types (Array[String]): Filters for work of the provided types. allowing: ['issue', 'ticket', 'task']
Past Actions:
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
Output: {"result": [('owned_by', ['who_am_i']), ('issue.priority', ['p0'])], "missing_action": ""}
""",
"""
Directive:Get all P0 issues owned by me
Tool: works_list with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- created_by (Array[String]): Filters for work created by any of these users.
- owned_by (Array[String]): Filters for work owned by any of these users.
- issue.rev_orgs (Array[String]): Filters for issues with any of the provided Rev organizations
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ['p0', 'p1', 'p2', 'p3']
- limit (Number): The maximum number of works to return.
- stage.name (Array[String]): Filters for records in the provided stage(s) by name
- ticket.needs_response (Boolean): Filters for tickets that need a response
- ticket.severity (Array[String]): Filters for tickets with any of the provided severities allowing: ['blocker', 'high', 'low', 'medium']
- ticket.source_channel (Array[String]): Filters for tickets with any of the provided source channels allowing: []
- types (Array[String]): Filters for work of the provided types. allowing: ['issue', 'ticket', 'task']
Past Actions:
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
Output: {"result": [('owned_by', ['who_am_i']), ('issue.priority', ['p0'])], "missing_action": ""}
""",
"""
Directive:Get all my P0
Tool: works_list with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- created_by (Array[String]): Filters for work created by any of these users.
- owned_by (Array[String]): Filters for work owned by any of these users.
- issue.rev_orgs (Array[String]): Filters for issues with any of the provided Rev organizations
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ['p0', 'p1', 'p2', 'p3']
- limit (Number): The maximum number of works to return.
- stage.name (Array[String]): Filters for records in the provided stage(s) by name
- ticket.needs_response (Boolean): Filters for tickets that need a response
- ticket.severity (Array[String]): Filters for tickets with any of the provided severities allowing: ['blocker', 'high', 'low', 'medium']
- ticket.source_channel (Array[String]): Filters for tickets with any of the provided source channels allowing: []
- types (Array[String]): Filters for work of the provided types. allowing: ['issue', 'ticket', 'task']
Past Actions:
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
Output: {"result": [('owned_by', ['who_am_i']), ('issue.priority', ['p0'])], "missing_action": ""}
""",
"""
Directive:Get all my high priority issues in backlog
Tool: get_backlog_items with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- created_by (Array[String]): Filters for work created by any of these users.
- owned_by (Array[String]): Filters for work owned by any of these users.
- issue.rev_orgs (Array[String]): Filters for issues with any of the provided Rev organizations
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ['p0', 'p1', 'p2', 'p3']
- limit (Number): The maximum number of works to return.
- stage.name (Array[String]): Filters for records in the provided stage(s) by name
- ticket.needs_response (Boolean): Filters for tickets that need a response
- ticket.severity (Array[String]): Filters for tickets with any of the provided severities allowing: ['blocker', 'high', 'low', 'medium']
- ticket.source_channel (Array[String]): Filters for tickets with any of the provided source channels allowing: []
- types (Array[String]): Filters for work of the provided types. allowing: ['issue', 'ticket', 'task']
Past Actions:
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
Output: {"result": [('owned_by', ['who_am_i']), ('issue.priority', ['high'])], "missing_action": ""}
""",
"""
Directive:Summarize the high severity tickets coming in from slack from customer Cust123
Tool: summarize_objects with args:
- objects (Array[Object]): List of objects to summarize
Past Actions:
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
Output: {"result": [], "missing_action": "Get high severity tickets coming in from slack from customer Cust123"}
""",
"""
Directive: Add my P0 issues to the current sprint 
Tool: add_work_to_sprint with args:
- objects (Array[Object]): List of objects to add to the sprint
- sprint (String): The sprint to add the objects to
Past Actions:
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
Output: {"result": [], "missing_action": "Get my P0 issues"}
""",
"""
Directive: Add my P0 issues to the current sprint 
Tool: add_work_to_sprint with args:
- objects (Array[Object]): List of objects to add to the sprint
- sprint (String): The sprint to add the objects to
Past Actions:
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
- Get my P0 issues: works_list
Output: {"result": [("objects", "works_list"), ("sprint", "get_sprint_id")], "missing_action": ""}
""",
"""
Directive: Get high severity tickets from the customer UltimateCustomer
Tool: works_list with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- created_by (Array[String]): Filters for work created by any of these users.
- owned_by (Array[String]): Filters for work owned by any of these users.
- issue.rev_orgs (Array[String]): Filters for issues with any of the provided Rev organizations
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ['p0', 'p1', 'p2', 'p3']
- limit (Number): The maximum number of works to return.
- stage.name (Array[String]): Filters for records in the provided stage(s) by name
- ticket.needs_response (Boolean): Filters for tickets that need a response
- ticket.severity (Array[String]): Filters for tickets with any of the provided severities allowing: ['blocker', 'high', 'low', 'medium']
- ticket.source_channel (Array[String]): Filters for tickets with any of the provided source channels allowing: []
- types (Array[String]): Filters for work of the provided types. allowing: ['issue', 'ticket', 'task']
Past Actions:
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
Output: {"result": [], "missing_action": "Get objects related to customer UltimateCustomer"}
""",
"""
Directive: Get high severity tickets from the customer UltimateCustomer
Tool: works_list with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- created_by (Array[String]): Filters for work created by any of these users.
- owned_by (Array[String]): Filters for work owned by any of these users.
- issue.rev_orgs (Array[String]): Filters for issues with any of the provided Rev organizations
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ['p0', 'p1', 'p2', 'p3']
- limit (Number): The maximum number of works to return.
- stage.name (Array[String]): Filters for records in the provided stage(s) by name
- ticket.needs_response (Boolean): Filters for tickets that need a response
- ticket.severity (Array[String]): Filters for tickets with any of the provided severities allowing: ['blocker', 'high', 'low', 'medium']
- ticket.source_channel (Array[String]): Filters for tickets with any of the provided source channels allowing: []
- types (Array[String]): Filters for work of the provided types. allowing: ['issue', 'ticket', 'task']
Past Actions:
- Get objects related to customer UltimateCustomer: search_objects_by_name
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
Output: {"result": [("applies_to_part", "search_objects_by_name"), ("ticket.severity", "high") ], "missing_action": ""}
""",
"""
Directive: Get all high severity tickets coming in from slack from customer Cust123
Tool: works_list with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- created_by (Array[String]): Filters for work created by any of these users.
- owned_by (Array[String]): Filters for work owned by any of these users.
- issue.rev_orgs (Array[String]): Filters for issues with any of the provided Rev organizations
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ['p0', 'p1', 'p2', 'p3']
- limit (Number): The maximum number of works to return.
- stage.name (Array[String]): Filters for records in the provided stage(s) by name
- ticket.needs_response (Boolean): Filters for tickets that need a response
- ticket.severity (Array[String]): Filters for tickets with any of the provided severities allowing: ['blocker', 'high', 'low', 'medium']
- ticket.source_channel (Array[String]): Filters for tickets with any of the provided source channels allowing: []
- types (Array[String]): Filters for work of the provided types. allowing: ['issue', 'ticket', 'task']
Past Actions:
- Get objects related to customer Cust123: search_objects_by_name
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
Output: {"result": [("applies_to_part", "search_objects_by_name"), ("ticket.severity", "high") ], "missing_action": ""}
""",
"""
Directive: Get all high severity tickets coming in from slack from customer Cust123
Tool: works_list with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- created_by (Array[String]): Filters for work created by any of these users.
- owned_by (Array[String]): Filters for work owned by any of these users.
- issue.rev_orgs (Array[String]): Filters for issues with any of the provided Rev organizations
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ['p0', 'p1', 'p2', 'p3']
- limit (Number): The maximum number of works to return.
- stage.name (Array[String]): Filters for records in the provided stage(s) by name
- ticket.needs_response (Boolean): Filters for tickets that need a response
- ticket.severity (Array[String]): Filters for tickets with any of the provided severities allowing: ['blocker', 'high', 'low', 'medium']
- ticket.source_channel (Array[String]): Filters for tickets with any of the provided source channels
- types (Array[String]): Filters for work of the provided types. allowing: ['issue', 'ticket', 'task']
Past Actions:
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
Output: {"result": [], "missing_action": "Get objects related to customer Cust123"}
""",
"""
Directive: Get action items from T
Tool: get_action_items_from_text with args:
- text (String): The text from which the actionable insights need to be created
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result": [("text", "T")], "missing_action": ""}
""",
"""
Directive: Create action items from meeting transcript T
Tool: get_action_items_from_text with args:
- text (String): The text from which the actionable insights need to be created
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result": [("text", "T")], "missing_action": ""}
""",
"""
Directive: Create issues from summarized work items
Tool: get_action_items_from_text with args:
- text (String): The text from which the actionable insights need to be created
Past Actions:
- Summarize the work items similar to TKT-123: summarize_objects
- Get all work items similar to TKT-123: get_similar_work_items
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result": [("text", "summarize_objects")], "missing_action": ""}
""",
"""
Directive: Create issues from summarized work items similar to TKT-123
Tool: get_action_items_from_text with args:
- text (String): The text from which the actionable insights need to be created
Past Actions:
- Get all work items similar to TKT-123: get_similar_work_items
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result": [], "missing_action": "Summarize the work items similar to TKT-123"}
""",
"""
Directive: Create issues from summarized work items similar to TKT-123
Tool: get_action_items_from_text with args:
- text (String): The text from which the actionable insights need to be created
Past Actions:
- Summarize the work items similar to TKT-123: summarize_objects
- Get all work items similar to TKT-123: get_similar_work_items
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result": [("text", "summarize_objects")], "missing_action": ""}
""",
"""
Directive: Create issues from summarized work items similar to TKT-123
Tool: get_action_items_from_text with args:
- text (String): The text from which the actionable insights need to be created
Past Actions:
- Summarize the work items similar to TKT-123: summarize_objects
- Get all work items similar to TKT-123: get_similar_work_items
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result": [("text", "summarize_objects")], "missing_action": ""}
""",
"""
Directive: Create issues from summarized work items
Tool: get_action_items_from_text with args:
- text (String): The text from which the actionable insights need to be created
Past Actions:
- Summarize the work items similar to TKT-123: summarize_objects
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result": [("text", "summarize_objects")], "missing_action": ""}
""",
"""
Directive: Add my prioritized P0 issues to the current sprint 
Tool: add_work_to_sprint with args:
- objects (Array[Object]): List of objects to add to the sprint
- sprint (String): The sprint to add the objects to
Past Actions:
- Prioritize my P0 issues: prioritize_objects
- Get my P0 issues: works_list
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
Output: {"result": [("objects", "prioritize_objects"), ("sprint", "get_sprint_id")], "missing_action": ""}
""",
"""
Directive:Summarize the high severity tickets coming in from slack from customer Cust123
Tool: summarize_objects with args:
- objects (Array[Object]): List of objects to summarize
Past Actions:
- Get the high severity tickets coming in from slack from customer Cust123: get_high_severity_tickets: works_list
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
Output: {"result": [("objects", "works_list")], "missing_action": ""}
""",
"""
Directive: Summarize the work items similar to TKT-123
Tool:summarize_objects with args:
- objects (Array[Object]):The objects to summarize
Past Actions:
- Prioritize the work items similar to TKT-123: prioritize_work_items
- Get all similar work items to TKT-123: get_similar_work_items
- the ID of the current User: who_am_i
- the current sprint:get_current_sprint
Output:{"result":[("objects","get_similar_work_items")],"missing_action":""}
""",
"""
Directive: Get my P0 issues on part DEV-PART-123
Tool:get_items with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- owned_by (Array[String]): Filters for work owned by any of these users
- created_by (Array[String]): Filters for work created by any of these users
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"]
Past Actions:
- the ID of the current User: who_am_i
Output:{"result": [("owned_by",["who_am_i"]),("issue.priority",["p0"]),("applies_to_part",["DEV-PART-123"])],"missing_action": ""}
""",
"""
Directive: Summarize my P0 issues
Tool:summarize_objects with args:
- objects (Array[Object]):The objects to summarize
Past Actions:
- the ID of the current User: who_am_i
Output: {"result": [],"missing_action": "Get my P0 issues"}
""",
"""
Directive: Prioritize my P0 issues
Tool:prioritize_objects with args:
- objects (Array[Object]): The objects to summarize
Past Actions:
- the ID of the current User: who_am_i
Output: {"result": [],"missing_action": "Get my P0 issues"}
""",
"""
Directive: Send an email to User-123 regarding issue I-123.
Tool: send_email with args:
- to (String): The email address of the recipient
- from (String): The email address of the sender
- subject (String): The subject of the email
- body (String): The body of the email
Past Actions:
- Reutrn user details of User-123: get_user_details
Output: {"result": [], "missing_action": "Summarize issue I-123"}
""",
"""
Directive: Get all issues
Tool:get_items with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- owned_by (Array[String]): Filters for work owned by any of these users
- created_by (Array[String]): Filters for work created by any of these users
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"]
Past Actions:
Output:{"result": [],"missing_action": ""}
""",
"""
Directive: Send an email to User-123 regarding issue I-123.
Tool: send_email with args:
- to (String): The email address of the recipient
- subject (String): The subject of the email
- body (String): The body of the email
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result": [], "missing_action": "Get user details of User-123"}
""",
"""
Directive:Get my P0 issues on all objects containing the word "AI"
Tool:get_items with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- owned_by (Array[String]): Filters for work owned by any of these users
- created_by (Array[String]): Filters for work created by any of these users
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"]
Past Actions:
- the ID of the current User: who_am_i
Output:{"result": [],"missing_action": "Get all parts containing the word 'AI'"}
""",
"""
Directive: Get my P0 issues on all objects containing the word "AI"
Tool:get_items with args:
- applies_to_part (String): Filters for work belonging to any of the provided parts.
- owned_by (String): Filters for work owned by any of these users
- created_by (String): Filters for work created by any of these users
- issue.priority (String): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"]
Past Actions:
- Get all parts containing the word "AI": get_objects_by_keyword
- the ID of the current User: who_am_i
Output:{"result": [("owned_by","who_am_i"),("issue.priority","p0"),("applies_to_part","get_objects_by_keyword")],"missing_action": ""}
""",
"""
Directive: Send an email to User-123 regarding issue I-123.
Tool: send_email with args:
- to (String): The email address of the recipient
- from (String): The email address of the sender
- subject (String): The subject of the email
- body (String): The body of the email
Past Actions:
- Get user details of User-123: get_user_details
- Summarize issue I-123: summarize_objects
- Returns the ID of the current User: who_am_i
Output: {"result": [("to", "get_user_details"), ("from", "who_am_i"), ("subject", "Regarding issue I-123"), ("body", "summarize_objects")],"missing_action": ""}
""",
"""
Directive: Dial up the user User-123
Tool: call_user with args:
- contact_details (Object): The contact details of the user
Past Actions:
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [], "missing_action": "Get user details of User-123"}
""",
"""
Directive: Dial up the user User-123
Tool: call_user with args:
- contact_details (Object): The contact details of the user
Past Actions:
- Get user details of User-123: get_user_details
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [("contact_details", "get_user_details")], "missing_action": ""}
""",
"""
Directive: Pay user User-123 his balance amount
Tool: pay_user with args:
- contact_details (Object): The contact details of the user
- amount (Number): The amount to pay the user
Past Actions:
- Get balance amount of User-123: get_user_balance
- Get user details of User-123: get_user_details
- the current sprint: get_current_sprint
Output: {"result": [("contact_details", "get_user_details"), ("amount", "get_user_balance")], "missing_action": ""}
""",
"""
Directive: Pay user User-123 his balance amount
Tool: pay_user with args:
- contact_details (Object): The contact details of the user
- amount (Number): The amount to pay the user
Past Actions:
- Returns user details of User-123: get_user_details
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [], "missing_action": "Get balance amount of User-123"}
""",
"""
Directive: Pay user User-123 $100
Tool: pay_user with args:
- contact_details (Object): The contact details of the user
- amount (Number): The amount to pay the user
Past Actions:
- Returns user details of User-123: get_user_details
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [("contact_details", "get_user_details"), ("amount", 100)], "missing_action": ""}
""",
"""
Directive: Close all unresolved issues
Tool: close_issues with args:
- issues (Array[Object]): List of issues to close
Past Actions:
Output: {"result": [], "missing_action": "Get the list of unresolved issues"}
""",
"""
Directive: Move all tasks to the next sprint
Tool: move_issues with args:
- issues (Array[Object]): List of issues to move
- destination_sprint (String): The sprint to move the issues to
Past Actions:
- the ID of the current User: who_am_i
- the next sprint: get_next_sprint
Output: {"result": [], "missing_action": "Get the list of all tasks"}
""",
"""
Directive: Move all unresolved issues to the next sprint
Tool: move_issues with args:
- issues (Array[Object]): List of issues to move
- destination_sprint (String): The sprint to move the issues to
Past Actions:
- Returns List of unresolved issues: get_unresolved_issues
- Get the ID of the current User: who_am_i
- Return the next sprint: get_next_sprint
Output: {"result": [("issues", "get_unresolved_issues"), ("destination_sprint", "get_next_sprint")], "missing_action": ""}
""",
"""
Directive: Get my issues on all objects made by MAN-123
Tool:get_items with args:
- applies_to_part (String): Filters for work belonging to any of the provided parts.
- owned_by (String): Filters for work owned by any of these users
- created_by (String): Filters for work created by any of these users
- issue.priority (String): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"]
Past Actions:
- Returns all parts containing the word "AI": get_objects_by_keyword
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output:{"result": [("owned_by","who_am_i"),("created_by","MAN-123")],"missing_action": ""}
""",
"""
Directive: Get my issues on all objects created by MAN-123
Tool:get_items with args:
- applies_to_part (String): Filters for work belonging to any of the provided parts.
- owned_by (String): Filters for work owned by any of these users
- created_by (String): Filters for work created by any of these users
- issue.priority (String): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"]
Past Actions:
- Returns all parts containing the word "AI": get_objects_by_keyword
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output:{"result": [("owned_by","who_am_i"),("created_by","MAN-123")],"missing_action": ""}
""",
"""
Directive: Get my issues on all objects made by my Manager
Tool:get_items with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- owned_by (Array[String]): Filters for work owned by any of these users
- created_by (Array[String]): Filters for work created by any of these users
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"]
Past Actions:
- Get all parts containing the word "AI": get_objects_by_keyword
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output:{"result": [], "missing_action": "Get user details of my Manager"}
""",
"""
Directive: Get issues made by users in my team
Tool:get_items with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- owned_by (Array[String]): Filters for work owned by any of these users
- created_by (Array[String]): Filters for work created by any of these users
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"]
Past Actions:
Output:{"result": [],"missing_action": "Get user details of my team"}
""",
"""
Directive: Get issues made by users U-123 and U-456
Tool:get_items with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- owned_by (Array[String]): Filters for work owned by any of these users
- created_by (Array[String]): Filters for work created by any of these users
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"]
Past Actions:
- Get all parts containing the word "AI": get_objects_by_keyword
- the ID of the current User: who_am_i
Output:{"result": [("created_by", ["U-123", "U-456"])],"missing_action": ""}
""",
"""
Directive: Get issues owned by users U-123 and U-456 and with priorities P0 and P1
Tool:get_work_items with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- owned_by (Array[String]): Filters for work owned by any of these users
- created_by (Array[String]): Filters for work created by any of these users
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"]
Past Actions:
- Get all parts containing the word "AI": get_objects_by_keyword
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output:{"result": [("created_by", ["U-123", "U-456"]), ("issue.priority",["p0", "p1"])],"missing_action": ""}
""",
"""
Directive: Summarize my open tasks
Tool: summarize_objects with args:
- objects (Array[Object]): The objects to summarize
Past Actions:
Output: {"result": [], "missing_action": "Get my open tasks"}
""",
"""
Directive: Assign a QA tester to Project P-789
Tool: assign_user with args:
- task_key (String): The key of the task to assign
- user_key (Array[String]): The key of the QA tester to assign to the issue
Past Actions:
- List of available QA testers: get_available_qa_testers
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [("task_key", "P-789"), ("user_key", "get_available_qa_testers")], "missing_action": ""}
""",
"""
Directive: Create a new high-priority issue from action item creted from T in the current backlog
Tool:create_issue with args:
- summary (String): A brief, one-sentence description of the issue
- description (String): A detailed description of the issue
- priority (String): Priority of the issue allowing: ["High", "Medium", "Low", "Blocker"]
- backlog (Array[String]): The backlog to add the issue to
Past Actions:
- Get list of available priorities: get_available_priorities
- Get action items created from T: get_action_items_from_text
- Get Current backlog: get_backlog_issues
- Get the ID of the current User: who_am_i
Output: {"result": [("summary", ""), ("description", ""), ("priority", "High"), ("backlog", "get_backlog_issues")], "missing_action": ""}
""",
"""
Directive: Create an action items created from transcript T
Tool: create_actionable_tasks_from_text with args:
- text (String):The text from which the actionable insights need to be created
Past Actions:
- List of available priorities: get_available_priorities
- Get action items created from T: get_action_items_from_text
- Get Current backlog: get_backlog_issues
- Get the ID of the current User: who_am_i
Output: {"result": [("text", "T")], "missing_action": ""}
""",
"""
Directive: Assign developers to all tasks in the backlog
Tool: assign_users with args:
- tasks (Array[Object]): List of tasks to assign
- users (Array[String]): List of developers to assign to the tasks
Past Actions:
- List of developers: get_available_developers
- List of tasks in the backlog: get_backlog_tasks
- the ID of the current User: who_am_i
Output: {"result": [("tasks", "get_backlog_tasks"), ("users", "get_available_developers")], "missing_action": ""}
""",
"""
Directive: Assign developers D-123, D-234 and Bob to unresolved tasks in the backlog
Tool: assign_users with args:
- tasks (Array[Object]): List of tasks to assign
- users (Array[String]): List of developers to assign to the tasks
Past Actions:
- List of developers: get_available_developers
- Get the list of unresolved tasks in the backlog: get_backlog_tasks
- the ID of the current User: who_am_i
Output: {"result": [("tasks", "get_backlog_tasks"), ("users", ["D-123", "D-234", "Bob"])], "missing_action": ""}
""",
"""
Directive: Summarize the tasks completed by my team
Tool: summarize_objects with args:
- objects (Array[Object]): The objects to summarize
Past Actions:
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [], "missing_action": "Get completed tasks by my team"}
""",
"""
Directive: Create a new issue with the summary "Fix bug in AI" and the description "This is a bug in AI"
Tool:create_issue with args:
- summary (String): A brief, one-sentence description of the issue
- description (String): A detailed description of the issue
- priority (String): Priority of the issue allowing: ["High", "Medium", "Low", "Blocker"]
- backlog (String): The backlog to add the issue to
Past Actions:
- List of available priorities: get_available_priorities
- Get the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [("summary", "Fix bug in AI"), ("description", "This is a bug in AI")], "missing_action": ""}
""",
"""
Directive: Create a new issue with the summary "Fix bug in AI"
Tool:create_issue with args:
- summary (String): A brief, one-sentence description of the issue
- description (String): A detailed description of the issue
- priority (String): Priority of the issue allowing: ["High", "Medium", "Low", "Blocker"]
- backlog (String): The backlog to add the issue to
Past Actions:
- List of available priorities: get_available_priorities
- Get the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [("summary", "Fix bug in AI"), ("description", "This is an issue to fix a bug in AI")], "missing_action": ""}
""",
"""
Directive: Close all unresolved support tickets
Tool: close_issues with args:
- issues (Array[Object]): List of issues to close
Past Actions:
- Get the list of all users: get_all_users
Output: {"result": [], "missing_action": "Get the list of unresolved support tickets"}
""",
"""
Directive: Close all unresolved bugs in the system
Tool: close_issues with args:
- issues (Array[Object]): List of issues to close
- issue.type (String): Filters for issues by the type ["Bug", "Feature", "Optimisation"]
- issue.status (String): Filters for issues by the status allowing ["Unresolved", "Resolved", "Abandoned"]
Past Actions:
- Get the list of all bugs: get_all_bugs
Output: {"result": [("issues", "get_all_bugs"), ("issue.type", "Bug"), ("issue.status", "Unresolved")], "missing_action": ""}
""",
"""
Directive: Close all unresolved bugs in the system
Tool: close_issues with args:
- issues (Array[Object]): List of issues to close
- issue.type (Array[String]): Filters for issues by the type ["Bug", "Feature", "Optimisation"]
- issue.status (String): Filters for issues by the status allowing ["Unresolved", "Resolved", "Abandoned"]
Past Actions:
- Get the list of all bugs: get_all_bugs
Output: {"result": [("issues", "get_all_bugs"), ("issue.type", ["Bug"]), ("issue.status", "Unresolved")], "missing_action": ""}
""",
"""
Directive: Assign a UI/UX designer to tasks in sprint S-456
Tool: assign_user with args:
- task_keys (Array[String]): The keys of the tasks to assign
- user_key (Array[String]): The key of the UI/UX designer to assign to the issue
Past Actions:
- List of available UI/UX designers: get_available_workers
- Get tasks in sprint S-456: get_tasks_in_sprint
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [("task_keys", "get_tasks_in_sprint"), ("user_key", "get_available_workers")], "missing_action": ""}
""",
"""
Directive: Assign a UI/UX designer to tasks in sprint S-456
Tool: assign_user with args:
- task_keys (Array[String]): The keys of the tasks to assign
- user_key (Array[String]): The key of the UI/UX designer to assign to the issue
Past Actions:
- List of available UI/UX designers: get_available_workers
- the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [], "missing_action": "Get tasks in sprint S-456"}
""",
"""
Directive: Create a new Blocking issue with the summary "Implement feature X" and the description "Details about implementing feature X"
Tool:create_issue with args:
- summary (String): A brief, one-sentence description of the issue
- description (String): A detailed description of the issue
- priority (String): Priority of the issue allowing: ["High", "Medium", "Low", "Blocker"]
- backlog (String): The backlog to add the issue to
Past Actions:
- List of available priorities: get_available_priorities
- Get the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [("summary", "Implement feature X"), ("description", "Details about implementing feature X"), ("priority", "Blocker")], "missing_action": ""}
""",
"""
Directive: Create a new Blocking issue with the summary 'Implement feature X'"
Tool:create_issue with args:
- summary (String): A brief, one-sentence description of the issue
- description (String): A detailed description of the issue
- priority (String): Priority of the issue allowing: ["High", "Medium", "Low", "Blocker"]
- backlog (String): The backlog to add the issue to
Past Actions:
- List of available priorities: get_available_priorities
- Get the ID of the current User: who_am_i
- the current sprint: get_current_sprint
Output: {"result": [("summary", "Implement feature X"), ("description", "This is a blocking issue to implement feature X"), ("priority", "Blocker")], "missing_action": ""}
""",
"""
Directive: Commit my changes to the current repository on a branch named "feature-123"
Tool:commit_changes with args:
- repository (String): The repository to commit to
- description (String): A detailed description of the commit
- branch (String): The branch to commit to
Past Actions:
- Get the changes summary: get_changes_summary
- Get the name of the current branch: get_current_branch
- Get the name current repository: get_current_repository
Output: {"result": [("repository", "get_current_repository"), ("description", "get_changes_summary"), ("branch", "feature-123")], "missing_action": ""}
""",
"""
Directive: Commit my changes to the current repository on a branch named "feature-123"
Tool:commit_changes with args:
- repository (String): The repository to commit to
- description (String): A detailed description of the commit
- branch (String): The branch to commit to
Past Actions:
- Get the name of the current branch: get_current_branch
- Get the name current repository: get_current_repository
Output: {"result": [], "missing_action": "Get the commit change description summary"}
""",
"""
Directive: Move all files with names starting with "feature-file" to the folder "folder-123"
Tool:move_files with args:
- files (Array[String]): The files to move
- destination (String): The destination folder
Past Actions:
- Get files with names starting with "feature-file": get_files_by_name
- Get the name of the current branch: get_current_branch
- Get the name current repository: get_current_repository
Output: {"result": [("files", "get_files_by_name"), ("destination", "folder-123")], "missing_action": ""}
""",
"""
Directive: Move files "file-123", "file-ABC" and "file-678" to the folder "folder-123"
Tool:move_files with args:
- files (Array[String]): The files to move
- destination (String): The destination folder
Past Actions:
- Get files with names starting with "feature-file": get_files_by_name
- Get the name of the current branch: get_current_branch
- Get the name current repository: get_current_repository
Output: {"result": [("files", ["file-123", "file-ABC", "file-678"]), ("destination", "folder-123")], "missing_action": ""}
""",
"""
Directive: Move all files with names starting with "feature-file" to the folder "folder-123"
Tool:move_files with args:
- files (Array[String]): The files to move
- destination (String): The destination folder
- delete (Boolean): Whether to delete the files after moving them
- overwrite (Boolean): Whether to overwrite the files if they already exist at destination
Past Actions:
- Get the id of the current user: who_am_i
- Get the current sprint: get_current_sprint
- Get the name of the current branch: get_current_branch
- Get the name current repository: get_current_repository
Output: {"result": [], "missing_action": "Get all files with names starting with 'feature-file'"}
""",
"""
Directive: Get all files with names starting with "feature-file"
Tool:search_files_by_name with args:
- name (String): The name of the files to get
Past Actions:
- Get the id of the current user: who_am_i
- Get the current sprint: get_current_sprint
- Get the name of the current branch: get_current_branch
- Get the name current repository: get_current_repository
Output: {"result": [("name", "feature-file")], "missing_action": ""}
""",
"""
Directive: Get the current sprint
Tool:get_current_sprint with args:
Past Actions:
- Get the id of the current user: who_am_i
- Get the id of the current sprint: get_current_sprint
- Returns the name of the current branch: get_current_branch
- Returns the name current repository: get_current_repository
Output: {"result": [], "missing_action": ""}
""",
"""
Directive: Summarize all the completed tasks from the last sprint
Tool: summarize_objects with args:
- objects (Array[Object]): The objects to summarize
Past Actions:
- Get the id of the current user: who_am_i
- Get the id of the current sprint: get_current_sprint
- Get the name of the current branch: get_current_branch
Output: {"result": [], "missing_action": "Get all the completed tasks from the last sprint"}
""",
"""
Directive: Summarize all the completed tasks from the last sprint
Tool: summarize_objects with args:
- objects (Array[Object]): The objects to summarize
Past Actions:
- Get the id of the current user: who_am_i
- Get the id of the current sprint: get_current_sprint
- Returns all completed tasks from the last sprint: get_completed_tasks
- Get last sprint: get_last_sprint
- Get the name of the current branch: get_current_branch
Output: {"result": [("objects", "get_completed_tasks")], "missing_action": ""}
""",
"""
Directive:Summarize my P0 issues
Tool: summarize_objects with args:
- objects (Array[Object]): List of objects to summarize
Past Actions:
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
Output: {"result": [], "missing_action": "Get my P0 issues"}
""",
"""
Directive:Get all high severity tickets coming in from slack from customer Cust123
Tool: works_list with args:
- applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
- created_by (Array[String]): Filters for work created by any of these users
- owned_by (Array[String]): Filters for work owned by any of these users
- issue.rev_orgs (Array[String]): Filters for issues with any of the provided Rev organizations
- issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ['p0', 'p1', 'p2', 'p3']
- limit (Number): The maximum number of works to return.
- stage.name (Array[String]): Filters for records in the provided stage(s) by name
- ticket.needs_response (Boolean): Filters for tickets that need a response
- ticket.severity (Array[String]): Filters for tickets with any of the provided severities allowing: ['blocker', 'high', 'low', 'medium']
- ticket.source_channel (String): Filters for tickets with any of the provided source channels
- types (String): Filters for work of the provided types allowing: ['issue', 'ticket', 'task']
Past Actions:
- Returns the ID of the current user: who_am_i
- Returns the ID of the current sprint: get_sprint_id
Output:{'result': [], 'missing_action': 'Get all objects related to customer Cust123'}
""",
"""
Directive: Assign tasks in the backlog to team members
Tool: assign_tasks with args:
- tasks (Array[Object]): List of tasks to assign
- team_members (Array[String]): List of team members to assign tasks to
Past Actions:
- Get the ID of the current User: who_am_i
- Get the backlog tasks: get_backlog_tasks
Output: {"result": [], "missing_action": "Get the list of team members"}
""",
"""
Directive: Assign tasks in the backlog to team members
Tool: assign_tasks with args:
- tasks (Array[Object]): List of tasks to assign
- team_members (Array[String]): List of team members to assign tasks to
Past Actions:
- Get the ID of the current User: who_am_i
- Get my team members: get_team_information
Output: {"result": [], "missing_action": "Get the list of backlog tasks"}
""",
"""
Directive: Assign tasks in the backlog to team members
Tool: assign_tasks with args:
- tasks (Array[Object]): List of tasks to assign
- team_members (Array[String]): List of team members to assign tasks to
Past Actions:
- Get the backlog tasks: get_tasks
- Get my team members: get_team_information
- Get the ID of the current User: who_am_i
Output: {"result": [("tasks", "get_tasks"), ("team_members", "get_team_information")], "missing_action": ""}
""",
"""
Directive: Create a report for all completed tasks in the last sprint
Tool: create_report with args:
- tasks (Array[Object]): List of tasks to include in the report
- sprint (String): The sprint for which the report is generated
Past Actions:
- Get the ID of the current User: who_am_i
- Get the completed tasks in the last sprint: get_task_info
- Get the last sprint: get_last_sprint
Output: {"result": [("tasks", "get_task_info"), ("sprint", "get_last_sprint")], "missing_action": ""}
""",
"""
Directive: Add the new task to my current project
Tool: add_task with args:
- task (Object): The task to be added to the project
- project (String): Project to which the task should be added
Past Actions:
- Create a new task with description D: create_task
- Get the ID of the current User: who_am_i
- Get my curretn project: get_projects
Output: {"result": [("project", "get_projects"), ("task", "create_task")], "missing_action": ""}
""",
"""
Directive: Add the new task with description D to my current project
Tool: add_task with args:
- task (Object): The task to be added to the project
- project (String): Project to which the task should be added
Past Actions:
- Get the ID of the current User: who_am_i
- Get my curretn project: get_projects
Output: {"result": [], "missing_action": "Create a new task with description D"}
""",
"""
Directive: Identify all my overdue tasks
Tool: filter_tasks with args:
- tasks (Array[Object]): List of tasks to filter
- status (Array[String]): The status of the tasks to filter allowing: ['Completed', 'In Progress', 'Overdue']
Past Actions:
- Get the ID of the current User: who_am_i
- Get all tasks assigned to me: get_my_assigned_tasks
Output: {"result": [("tasks", "get_my_assigned_tasks"), ("status", ["Overdue"])], "missing_action": ""}
""",
"""
Directive: Create a summary of all tasks completed by the team this month
Tool: summarize_tasks with args:
- tasks (Array[Object]): List of tasks to include in the summary
- time_period (String): The time period for the summary allowing: ["this_month", "last_month", "this_week", "last_week", "today"]
Past Actions:
- Get the ID of the current User: who_am_i
- Get the completed tasks for the team this month: get_task_information
Output: {"result": [("tasks", "get_task_information"), ("time_period", "this_month")], "missing_action": ""}
""",
"""
Directive: Update the priority of high-priority tasks
Tool: update_priority with args:
- tasks (Array[Object]): List of tasks to update
- new_priority (String): The new priority for the tasks allowing: ["high", "low", "medium"]
Past Actions:
- Get the ID of the current User: who_am_i
- Get all high-priority tasks: get_high_priority_tasks
Output: {"result": [("tasks", "get_high_priority_tasks"), ("new_priority", "high")], "missing_action": ""}
""",
"""
Directive: Find all tasks related to the 'Feature-123' feature
Tool: find_related_tasks with args:
- feature_name (String): Name of the feature to find related tasks
Past Actions:
- Get the ID of the current User: who_am_i
- Search for tasks related to 'Feature-123': search_related_tasks
Output: {"result": [("feature_name", "Feature-123")], "missing_action": ""}
""",
"""
Directive: Find all tasks related to the 'Feature-123','Feature-456' and 'Feature-789' feature
Tool: find_related_tasks with args:
- features (Array[String]): Name of the feature to find related tasks
- priority (String): The priority of the tasks to filter allowing: ["high", "low", "medium"]
- organization (String): The organization of the tasks to filter
- status (Array[String]): The status of the tasks to filter allowing: ["Completed", "In Progress", "Overdue"]
Past Actions:
- Get the ID of the current User: who_am_i
- Search for tasks related to 'Feature-123': search_related_tasks
Output: {"result": [("features", ["Feature-123", "Feature-456", "Feature-789"])], "missing_action": ""}
""",
"""
Directive: Find all high and medium tasks related to the 'Feature-123','Feature-456' and 'Feature-789' feature
Tool: find_related_tasks with args:
- features (Array[String]): Name of the feature to find related tasks
- priority (String): The priority of the tasks to filter allowing: ["high", "low", "medium"]
- organization (String): The organization of the tasks to filter
- status (Array[String]): The status of the tasks to filter allowing: ["Completed", "In Progress", "Overdue"]
Past Actions:
- Get the ID of the current User: who_am_i
- Search for tasks related to 'Feature-123': search_related_tasks
Output: {"result": [("features", ["Feature-123", "Feature-456", "Feature-789"]), ("priority", ['high', 'medium'])], "missing_action": ""}
""",
"""
Directive: Reassign unresolved issues to me and User-123
Tool: reassign_issues with args:
- issues (Array[Object]): List of issues to reassign
- new_assignees (Array[String]): List of users to assign the issues to
Past Actions:
- Get the ID of the current User: who_am_i
- Get all unresolved issues: get_issues_by_filter
Output: {"result": [("issues", "get_issues_by_filter"), ("new_assignees", ["who_am_i", "User-123"])], "missing_action": ""}
""",
"""
Directive: Reassign unresolved issues to me and User-123
Tool: reassign_issues with args:
- issues (Array[Object]): List of issues to reassign
- new_assignees (Array[String]): List of users to assign the issues to
Past Actions:
- Get the ID of the current User: who_am_i
Output: {"result": [], "missing_action": "Get unresolved issues"}
""",
"""
Directive: Reassign unresolved issues to me and and my team members
Tool: reassign_issues with args:
- issues (Array[Object]): List of issues to reassign
- new_assignees (Array[String]): List of users to assign the issues to
Past Actions:
- Get the ID of the current User: who_am_i
- Get my team members: get_team_information
Output: {"result": [], "missing_action": "Get unresolved issues"}
""",
"""
Directive: Reassign unresolved issues to me and and my team members
Tool: reassign_issues with args:
- issues (Array[Object]): List of issues to reassign
- new_assignees (Array[String]): List of users to assign the issues to
Past Actions:
- Get the ID of the current User: who_am_i
- Get all unresolved issues: get_issues_by_filter
Output: {"result": [], "missing_action": "Get my team members"}
""",
"""
Directive: Summarize unresolved issues given to me and and my team members
Tool: summarize_issues with args:
- issues (Array[Object]): List of issues to summarize
Past Actions:
- Get the ID of the current User: who_am_i
- Get all unresolved issues given to me and my team members: get_issues_by_filter
Output: {"result": [("issues", "get_issues_by_filter")], "missing_action": ""}
""",
"""
Directive: Summarize unresolved issues given to me and and my team members
Tool: summarize_issues with args:
- issues (Array[Object]): List of issues to summarize
Past Actions:
- Get the ID of the current User: who_am_i
- Get current_sprint: get_current_sprint
- Get all unresolved issues: get_issues_by_filter
Output: {"result": [], "missing_action": "Get all unresolved issues given to me and my team members"}
""",
"""
Directive: Prioritize unresolved issues given to me and and my team members
Tool: prioritize_issues with args:
- issues (Array[Object]): List of issues to prioritize
Past Actions:
- Get the ID of the current User: who_am_i
- Get current_sprint: get_current_sprint
- Get all unresolved issues: get_issues_by_filter
Output: {"result": [], "missing_action": "Get all unresolved issues given to me and my team members"}
""",
"""
Directive: Prioritize unresolved issues given to me and and my team members
Tool: prioritize_issues with args:
- issues (Array[Object]): List of issues to prioritize
Past Actions:
- Get the ID of the current User: who_am_i
- Get all unresolved issues given to me and my team members: get_issues_by_filter
Output: {"result": [("issues", "get_issues_by_filter")], "missing_action": ""}
""",
"""
Directive: Reassign unresolved issues to me and and my team members
Tool: assign_issues with args:
- issues (Array[Object]): List of issues to reassign
- assignees (Array[String]): List of users to assign the issues to
Past Actions:
- Get the ID of the current User: who_am_i
- Get my team members: get_team_information
- Get all unresolved issues: get_issues_by_filter
Output: {"result": [("issues", "get_issues_by_filter"), ("assignees", ["who_am_i", "get_team_information"])], "missing_action": ""}
""",
"""
Directive: Assign all unresolved issues to me
Tool: assign_issues with args:
- issues (Array[Object]): List of issues to assign
- assignees (Array[String]): List of users to assign the issues to
Past Actions:
- Get the ID of the current User: who_am_i
- Get my team members: get_team_information
- Get all unresolved issues: get_issues_by_filter
Output: {"result": [("issues", "get_issues_by_filter"), ("assignees", ["who_am_i"])], "missing_action": ""}
""",
"""
Directive: Generate a performance report for tasks completed in the last two sprints
Tool: generate_report with args:
- tasks (Array[Object]): List of tasks to include in the report
- include_details (Boolean): Whether to include detailed information in the report
- include_summary (Boolean): Whether to include summary information in the report
- rate_employee (Boolean): Whether to rate the employee in the report
Past Actions:
- Get the ID of the current User: who_am_i
- Get completed tasks in the last two sprints: get_task_information
Output: {"result": [("tasks", "get_task_information")], "missing_action": ""}
""",
"""
Directive: Identify tasks with overdue deadlines in the 'Marketing' project
Tool: fetch_tasks with args:
- tasks (Array[Object]): List of tasks to check for overdue status
- projects (Array[String]): Name of the projects to filter tasks
- status (Array[String]): The status of the tasks to filter allowing: ["Completed", "In Progress", "Overdue"]
Past Actions:
- Get the ID of the current User: who_am_i
- Get tasks in the 'Marketing' project: get_tasks_in_project
Output: {"result": [("projects", ['Marketing']), ("status", ["Overdue"])], "missing_action": ""}
""",
"""
Directive: Identify tasks with overdue deadlines in the 'Marketing' project
Tool: filter_tasks with args:
- tasks (Array[Object]): List of tasks to check for overdue status
- project (String): Name of the project to filter tasks
- status (Array[String]): The status of the tasks to filter allowing: ["Completed", "In Progress", "Overdue"]
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
- Call my manager: call_user
Output: {"result": [("project", "Marketing"), ("status", ["Overdue"])], "missing_action": ""}
""",
"""
Directive: Identify tasks with overdue deadlines in the 'Marketing' project
Tool: filter_tasks with args:
- tasks (Array[Object]): List of tasks to check for overdue status
- status (String): The status of the tasks to filter allowing: ["Completed", "In Progress", "Overdue"]
Past Actions:
- Get the ID of the current User: who_am_i
- Get tasks in the 'Marketing' project: get_tasks_in_project
Output: {"result": [("tasks", "get_tasks_in_project"), ("status", "Overdue")], "missing_action": ""}
""",
"""
Directive: Identify tasks with overdue deadlines in the 'Marketing' project
Tool: filter_tasks with args:
- tasks (Array[Object]): List of tasks to check for overdue status
- status (String): The status of the tasks to filter allowing: ["Completed", "In Progress", "Overdue"]
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
- Call my manager: call_user
Output: {"result": [], "missing_action": "Get tasks in the 'Marketing' project"}
""",
"""
Directive: Show a list of my tasks pending review by the project manager
Tool: get_info_on_tasks with args:
- assigned_to (Array[String]): List of users to the tasks are assigned to
- status (Array[String]): The status of the tasks to filter allowing: ["Completed", "In Progress", "Overdue"]
- priority (String): The priority of the tasks to filter allowing: ["High", "Medium", "Low", "Blocker"]
- reviewers (Array[String]): User ids of the reviewers
Past Actions:
- Get the ID of the current User: who_am_i
- Get the id of the project manager: get_user_details
Output: {"result": [("assigned_to", ["who_am_i"]), ("reviewer", ["get_user_details"])], "missing_action": ""}
""",
"""
Directive: Show a list of my tasks pending review by the project manager
Tool: get_info_on_tasks with args:
- assigned_to (String): User to whom the task is assigned to
- status (Array[String]): The status of the tasks to filter allowing: ["Completed", "In Progress", "Overdue"]
- priority (String): The priority of the tasks to filter allowing: ["High", "Medium", "Low", "Blocker"]
- reviewers (Array[String]): User ids of the reviewers
Past Actions:
- Get the ID of the current User: who_am_i
- Get the id of the project manager: get_user_details
Output: {"result": [("assigned_to", "who_am_i"), ("reviewer", ["get_user_details"])], "missing_action": ""}
""",
"""
Directive: Show a list of my tasks pending review by the project manager
Tool: get_info_on_tasks with args:
- tasks (Array[Object]): List of tasks to include in the pending review list
- assigned_to (Array[String]): List of users to the tasks are assigned to
- status (Array[String]): The status of the tasks to filter allowing: ["Completed", "In Progress", "Overdue"]
- priority (String): The priority of the tasks to filter allowing: ["High", "Medium", "Low", "Blocker"]
- reviewers (Array[String]): User ids of the reviewers
Past Actions:
- Get the ID of the current User: who_am_i
Output: {"result": [], "missing_action": "Get the user id of the project manager"}
""",
"""
Directive: Show a list of my tasks pending review
Tool: get_info_on_tasks with args:
- tasks (Array[Object]): List of tasks to include in the pending review list
- assigned_to (Array[String]): List of users to the tasks are assigned to
- status (Array[String]): The status of the tasks to filter allowing: ["Completed", "In Progress", "Overdue"]
- priority (String): The priority of the tasks to filter allowing: ["High", "Medium", "Low", "Blocker"]
- reviewers (Array[String]): User ids of the reviewers
Past Actions:
Output: {"result": [], "missing_action": "Get the user id of current user"}
""",
"""
Directive: Move all 'Bug' issues to the 'Bug-Fix' sprint
Tool: move_issues with args:
- issues (Array[Object]): List of issues to move
- destination_sprint (String): The sprint to move the issues to
Past Actions:
- Get the ID of the current User: who_am_i
- Get all 'Bug' issues: get_bug_issues
Output: {"result": [("issues", "get_bug_issues"), ("destination_sprint", "Bug-Fix")], "missing_action": ""}
""",
"""
Directive: Move issues I-1, I-2 and I-3 to the 'Bug-Fix' sprint
Tool: move_issues with args:
- issues (Array[Object]): List of issues to move
- destination_sprint (String): The sprint to move the issues to
Past Actions:
- Get the ID of the current User: who_am_i
- Get all 'Bug' issues: get_bug_issues
Output: {"result": [("issues", ["I-1", "I-2", "I-3"]), ("destination_sprint", "Bug-Fix")], "missing_action": ""}
""",
"""
Directive: Prioritize tasks related to customer feedback
Tool: prioritize_tasks with args:
- tasks (Array[Object]): List of tasks to prioritize
Past Actions:
- Get the ID of the current User: who_am_i
- Retrieve tasks related to customer feedback: fetch_feedback_tasks
Output: {"result": [("tasks", "fetch_feedback_tasks")], "missing_action": ""}
""",
"""
Directive: Prioritize tasks T-1, T-2 and T-54 related to customer feedback
Tool: prioritize_tasks with args:
- tasks (Array[Object]): List of tasks to prioritize
Past Actions:
- Get the ID of the current User: who_am_i
- Retrieve tasks related to customer feedback: fetch_feedback_tasks
Output: {"result": [("tasks", ["T-1", "T-2", "T-45"])], "missing_action": ""}
""",
"""
Directive: Reassign high-priority tasks to available team members
Tool: reassign_tasks with args:
- tasks (Array[Object]): List of high-priority tasks to reassign
- assignes (Array[String]): List of team members available for reassignment
Past Actions:
- Get the ID of the current User: who_am_i
- Retrieve high-priority tasks: fetch_org_tasks
- Identify available team members: get_available_users
Output: {"result": [("tasks", "fetch_org_tasks"), ("assignes", "get_available_users")], "missing_action": ""}
""",
"""
Directive: Generate a trend analysis report for Google Stadia's completion time
Tool: generate_trend_report with args:
- type (String): The type of the report to generate, allowing: ["completion_time", "completion_rate", "resource_utilization"]
- project_name (String): Name of the project for analysis
Past Actions:
- Get the ID of the current User: who_am_i
- Retrieve tasks for trend analysis: get_tasks_for_trend_analysis
Output: {"result": [("type", "completion_time"), ("project_name", "Google Stadia's")], "missing_action": ""}
""",
"""
Directive: Generate a trend analysis report for Project-A completion time
Tool: generate_trend_report with args:
- type (String): The type of the report to generate, allowing: ["completion_time", "completion_rate", "resource_utilization"]
- project_name (String): Name of the project for analysis
Past Actions:
- Get the ID of the current User: who_am_i
- Retrieve tasks for trend analysis: get_tasks_for_trend_analysis
Output: {"result": [("type", "completion_time"), ("project_name", "Project-A")], "missing_action": ""}
""",
"""
Directive: Find all tasks related to the 'New Feature' in the upcoming sprint
Tool: find_related_tasks with args:
- feature_name (String): Name of the feature to find related tasks
- sprint (String): The upcoming sprint for task search
Past Actions:
- Get the ID of the current User: who_am_i
- Get the upcoming sprint: get_upcoming_sprint
Output: {"result": [("feature_name", "New Feature"), ("sprint", "get_upcoming_sprint")], "missing_action": ""}
""",
"""
Directive: Close all tasks in Project-A with the 'urgent' and 'lax' priority
Tool: close_tasks with args:
- tasks (Array[Object]): List of tasks to close
- priority (Array[String]): The priority of tasks to close, allowing ["urgent", "lax", "tight"]
Past Actions:
- Get the ID of the current User: who_am_i
- Get all tasks in Project-A: get_tasks_in_project
Output: {"result": [("tasks", "get_tasks_in_project"), ("priority", ["urgent", "lax"])], "missing_action": ""}
""",
"""
Directive: Assign unresolved issues to team members for resolution
Tool: issue_assigner with args:
- issues (Array[Object]): List of issues to assign
- assignees (Array[String]): List of team members to assign issues
Past Actions:
- Get the ID of the current User: who_am_i
- Get unresolved issues: get_unresolved_issues
- Identify team members for assignment: get_team_members
Output: {"result": [("issues", "get_unresolved_issues"), ("assignees", "get_team_members")], "missing_action": ""}
""",
"""
Directive: Assign unresolved issues to me and my team members for resolution
Tool: issue_assigner with args:
- issues (Array[Object]): List of issues to assign
- assignees (Array[String]): List of team members to assign issues
Past Actions:
- Get the ID of the current User: who_am_i
- Get unresolved issues: get_unresolved_issues
Output: {"result": [], "missing_action": "Get ids of me and my team members"}
""",
"""
Directive: Assign unresolved issues to me and my team members for resolution
Tool: issue_assigner with args:
- issues (Array[Object]): List of issues to assign
- assignees (Array[String]): List of team members to assign issues
Past Actions:
- Get the ID of the current User: who_am_i
- Get unresolved issues: get_unresolved_issues
- Get me and my team members: get_team_members
Output: {"result": [("issues", "get_unresolved_issues"), ("assignees", "get_team_members")], "missing_action": ""}
""",
"""
Directive: Assign unresolved issues to U-123 and U-007 for resolution
Tool: issue_assigner with args:
- issues (Array[Object]): List of issues to assign
- assignees (Array[String]): List of team members to assign issues
Past Actions:
- Get the ID of the current User: who_am_i
- Get unresolved issues: get_unresolved_issues
- Identify team members for assignment: get_team_members
Output: {"result": [("issues", "get_unresolved_issues"), ("assignees", ["U-123", "U-007"])], "missing_action": ""}
""",
"""
Directive: Create a report summarizing tasks completed by John Doe
Tool: create_individual_report with args:
- team_member (String): Name of the team member for individual report
- start_date (String): The start date for the report
- end_date (String): The end date for the report
Past Actions:
- Get the ID of the current User: who_am_i
Output: {"result": [("team_member", "John Doe")], "missing_action": ""}
""",
"""
Directive: Move all tasks with the 'enhancement' label to the 'Next Release' sprint
Tool: move_tasks_to_sprint with args:
- tasks (Array[Object]): List of tasks to move
- destination_sprint (String): The sprint to move tasks to
- delete (Boolean): Whether to delete the tasks after moving them
Past Actions:
- Get the ID of the current User: who_am_i
- Retrieve tasks with 'enhancement' label: get_enhancement_tasks
Output: {"result": [("tasks", "get_enhancement_tasks"), ("label", "enhancement"), ("destination_sprint", "Next Release")], "missing_action": ""}
""",
"""
Directive: Move all tasks with the 'enhancement' label to the 'Next Release' sprint
Tool: move_tasks_to_sprint with args:
- tasks (Array[Object]): List of tasks to move
- destination_sprint_id (String): The id sprint to move tasks to
- delete (Boolean): Whether to delete the tasks after moving them
Past Actions:
- Get the ID of the current User: who_am_i
- Retrieve tasks with 'enhancement' label: get_enhancement_tasks
- Get the id of 'Next Release' sprint: get_sprint_id
Output: {"result": [("tasks", "get_enhancement_tasks"), ("destination_sprint", "get_sprint_id")], "missing_action": ""}
""",
"""
Directive: Move all tasks with the 'enhancement' label to the 'Next Release' sprint
Tool: move_tasks_to_sprint with args:
- tasks (Array[Object]): List of tasks to move
- label (String): The label to filter tasks
- destination_sprint_id (String): The id sprint to move tasks to
Past Actions:
- Get the ID of the current User: who_am_i
- Retrieve tasks with 'enhancement' label: get_enhancement_tasks
Output: {"result": [], "missing_action": "Get the id of 'Next Release' sprint"}
""",
"""
Directive: Identify and prioritize tasks with a due date within the next week
Tool: prioritize_due_within_week with args:
- tasks (Array[Object]): List of tasks to prioritize
Past Actions:
- Get the ID of the current User: who_am_i
- Retrieve tasks with due date within the next week: get_tasks_details
Output: {"result": [("tasks", "get_tasks_details")], "missing_action": ""}
""",
"""
Directive: Create a list of unresolved issues assigned to the current user
Tool: create_assigned_issues_list with args:
- issues (Array[Object]): List of unresolved issues assigned to the current user
Past Actions:
- Get the ID of the current User: who_am_i
- Retrieve unresolved issues assigned to the current user: get_assigned_unresolved_issues
Output: {"result": [("issues", "get_assigned_unresolved_issues")], "missing_action": ""}
""",
"""
Directive: Close all tasks in the 'Testing' phase
Tool: close_tasks_in_phase with args:
- tasks (Array[Object]): List of tasks to close
- reason (String): The reason for closing the tasks
- delete (Boolean): Whether to delete the tasks after closing them
Past Actions:
- Get the ID of the current User: who_am_i
- Retrieve tasks in the 'Testing' phase: get_tasks
Output: {"result": [("tasks", "get_testing_phase_tasks")], "missing_action": ""}
""",
"""
Directive: Move all high-priority tasks to the 'P1-HighPriority' project
Tool: move_tasks with args:
- tasks (Array[Object]): List of tasks to move
- destination_project (String): The project to move tasks to, e.g., "P1-HighPriority"
- notify (Boolean): Whether to notify the assignee of the task about the move
- reset_progress (Boolean): Whether to reset the progress of the tasks after moving them
Past Actions:
- Get the ID of the current User: who_am_i
- Retrieve high-priority tasks: fetch_tasks_by_property
- Get the 'P1-HighPriority' project: get_high_priority_project
Output: {"result": [("tasks", "fetch_tasks_by_property"), ("destination_project", "'P1-HighPriority'")], "missing_action": ""}
""",
"""
Directive: Mark tasks collected with a tag urgent for the upcoming sprint
Tool: prioritize_tagged_tasks with args:
- tasks (Array[Object]): List of tasks to prioritize
- tag (String): The tag to mark the tasks with allowing: ["urgent", "triage", "critical"]
- sprint (String): The sprint to prioritize the tasks for
Past Actions:
- Get the ID of the current User: who_am_i
- Collect tasks assigned to me: get_task_by_assignee
- Get the upcoming sprint: get_upcoming_sprint
Output: {"result": [("tasks", "get_task_by_assignee"), ("tag", "urgent"), ("sprint", "get_upcoming_sprint")], "missing_action": ""}
""",
"""
Directive: Retrieve details for TKT-789
Tool: get_issue_details with args:
- issue_key (String): The key of the issue to retrieve details for
Past Actions:
- Get the ID of the current User: who_am_i
Output: {"result":[("issue_key", "TKT-789")],"missing_action":""}
""",
"""
Directive: Assign all critical issues to DEV-TEAM
Tool: assign_issues with args:
- issues (Array[Object]): List of issues to assign
- assignee (String): The user to assign the issues to
Past Actions:
- Get the current sprint: get_current_sprint
Output: {"result":[],"missing_action":"Get the list of critical issues"}
""",
"""
Directive: Update status of tasks in the backlog to 'In Progress'
Tool: update_issue_status with args:
- issues (Array[Object]): List of issues to update
- new_status (String): The new status for the issues
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result":[],"missing_action":"Get the list of tasks in the backlog"}
""",
"""
Directive: Update status of my tasks in the backlog to 'In Progress'
Tool: update_issue_status with args:
- issues (Array[Object]): List of issues to update
- new_status (String): The new status for the issues
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result":[],"missing_action":"Get the list of my tasks in the backlog"}
""",
"""
Directive: Update status of tasks owned by me in the backlog to 'In Progress'
Tool: update_issue_status with args:
- issues (Array[Object]): List of issues to update
- new_status (String): The new status for the issues
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result":[],"missing_action":"Get the list of my tasks in the backlog"}
""",
"""
Directive: Update status of tasks in the backlog to 'In Progress'
Tool: update_issue_properties with args:
- issues (Array[Object]): List of issues to update
- new_status (String): The new status for the issues
- priority (String): The new priority for the issues allowing: ["high", "low", "medium"]
- sprint (String): The sprint to update the issues for
- notify (Boolean): Whether to notify the assignee of the issue about the update
Past Actions:
- Get the ID of the current User: who_am_i
- Get tasks in backlog: get_related_tasks
Output: {"result":[("issues", "get_related_tasks"), ("status", "In Progress")],"missing_action":""}
""",
"""
Directive: Update priority of my tasks in the backlog to HIGH
Tool: update_issue_properties with args:
- issues (Array[Object]): List of issues to update
- new_status (String): The new status for the issues
- priority (String): The new priority for the issues allowing: ["high", "low", "medium"]
- sprint (String): The sprint to update the issues for
- notify (Boolean): Whether to notify the assignee of the issue about the update
Past Actions:
- Get the ID of the current User: who_am_i
- Get my tasks in backlog: get_related_tasks
Output: {"result":[("issues", "get_related_tasks"), ("priority", "high")],"missing_action":""}
""",
"""
Directive: Update priority of tasks owned by me in the backlog to HIGH
Tool: update_issue_properties with args:
- issues (Array[Object]): List of issues to update
- new_status (String): The new status for the issues
- priority (String): The new priority for the issues allowing: ["high", "low", "medium"]
- sprint (String): The sprint to update the issues for
- notify (Boolean): Whether to notify the assignee of the issue about the update
Past Actions:
- Get the ID of the current User: who_am_i
- Get my tasks in backlog: get_related_tasks
Output: {"result":[("issues", "get_related_tasks"), ("priority", "high")],"missing_action":""}
""",
"""
Directive: Find unresolved bugs in the last release
Tool: find_issues with args:
- part_description (Array[String]): Part of the description of the issues to find
- organization (String): The organization of the issues to find
- name (String): The name of the issues to find
- issue_type (Array[String]): The type of issues to find allowing: ["Bug", "Feature Request", "Task"]
- status (Array[String]): The status of the issues to find allowing: ["Abandoned", "Resolved", "Unresolved"]
- release_version (String): The version of the release to search in
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result":[("issue_type", ["Bug"]), ("status", ["Unresolved"])],"missing_action":""}
""",
"""
Directive: Find unresolved requests for features in the last release
Tool: find_issues with args:
- part_description (Array[String]): Part of the description of the issues to find
- organization (String): The organization of the issues to find
- name (String): The name of the issues to find
- issue_type (Array[String]): The type of issues to find allowing: ["Bug", "Feature Request", "Task"]
- status (Array[String]): The status of the issues to find allowing: ["Abandoned", "Resolved", "Unresolved"]
- release_version (String): The version of the release to search in
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
- Get high-priority issues: get_high_priority_issues
- Get my manager issues: get_user_id
- Mail my manager: mail_user
- Get last release: get_last_release
Output: {"result":[("issue_type", ["Feature Request"]), ("status", ["Unresolved"]), ("release_version", "get_last_release")],"missing_action":""}
""",
"""
Directive: Find for feature requests that were unresolved in the last release
Tool: find_issues with args:
- part_description (Array[String]): Part of the description of the issues to find
- organization (String): The organization of the issues to find
- name (String): The name of the issues to find
- issue_type (Array[String]): The type of issues to find allowing: ["Bug", "Feature Request", "Task"]
- status (Array[String]): The status of the issues to find allowing: ["Abandoned", "Resolved", "Unresolved"]
- release_version (String): The version of the release to search in
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
- Get high-priority issues: get_high_priority_issues
- Get my manager issues: get_user_id
- Mail my manager: mail_user
- Get last release: get_last_release
Output: {"result":[("issue_type", ["Feature Request"]), ("status", ["Unresolved"]), ("release_version", "get_last_release")],"missing_action":""}
""",
"""
Directive: Find unresolved requests for features in the last release
Tool: find_issues with args:
- part_description (Array[String]): Part of the description of the issues to find
- organization (String): The organization of the issues to find
- name (String): The name of the issues to find
- issue_type (String): The type of issues to find allowing: ["Bug", "Feature Request", "Task"]
- status (Array[String]): The status of the issues to find allowing: ["Abandoned", "Resolved", "Unresolved"]
- release_version (String): The version of the release to search in
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
- Get high-priority issues: get_high_priority_issues
- Get my manager issues: get_user_id
- Mail my manager: mail_user
- Get last release: get_last_release
Output: {"result":[("issue_type", "Feature Request"), ("status", ["Unresolved"]), ("release_version", "get_last_release")],"missing_action":""}
""",
"""
Directive: Schedule a meeting for priority tasks today
Tool: schedule_meeting with args:
- meeting_date (String): The date for the meeting
- tasks (Array[Object]): List of tasks to discuss
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result":[],"missing_action":"Get the list of priority tasks"}
""",
"""
Directive: Schedule a meeting for priority tasks today
Tool: schedule_meeting with args:
- meeting_date (String): The date for the meeting
- tasks (Array[Object]): List of tasks to discuss
Past Actions:
- Get the ID of the current User: who_am_i
- Get high priority tasks: get_tasks
- Get the current sprint: get_current_sprint
Output: {"result":[("meeting_date","DATE_TODAY"), ("tasks", "get_tasks")],"missing_action":""}
""",
"""
Directive: Schedule a meeting for Task 1, Task 2 and Task 3 today
Tool: schedule_meeting with args:
- meeting_date (String): The date for the meeting
- tasks (Array[Object]): List of tasks to discuss
Past Actions:
- Get the ID of the current User: who_am_i
- Get high priority tasks: get_tasks
- Get the current sprint: get_current_sprint
Output: {"result":[("meeting_date","DATE_TODAY"), ("tasks", ["Task 1", "Task 2", "Task 3"])],"missing_action":""}
""",
"""
Directive: Create a report for completed user stories
Tool: generate_report with args:
- stories (Array[Object]): List of user stories to generate a report for
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result":[],"missing_action":"Get the list of completed user stories"}
""",
"""
Directive: Reassign unresolved issues to the support team
Tool: reassign_issues with args:
- issues (Array[Object]): List of issues to reassign
- new_assignee (String): The user to reassign the issues to
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result":[],"missing_action":"Get the list of unresolved issues"}
""",
"""
Directive: Reassign unresolved issues to the support team
Tool: reassign_issues with args:
- issues (Array[Object]): List of issues to reassign
- new_assignee (Array[String]): The users to reassign the issues to
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
- Get unresolved issues: get_unresolved_issues
Output: {"result":[],"missing_action":"Get the support team ids"}
""",
"""
Directive: Reassign unresolved issues to the support team
Tool: reassign_issues with args:
- issues (Array[Object]): List of issues to reassign
- new_assignee (Array[String]): The users to reassign the issues to
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
- Get team information: get_team_information
- Get unresolved issues: get_unresolved_issues
Output: {"result":[("issues", "get_unresolved_issues"), ("new_assignee", "get_team_information")],"missing_action":""}
""",
"""
Directive: Reassign issues that are unresolved to the support team
Tool: reassign_issues with args:
- issues (Array[Object]): List of issues to reassign
- new_assignee (Array[String]): The users to reassign the issues to
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
- Get team information: get_team_information
- Get unresolved issues: get_unresolved_issues
Output: {"result":[("issues", "get_unresolved_issues"), ("new_assignee", "get_team_information")],"missing_action":""}
""",
"""
Directive: Mark high-priority tasks as completed
Tool: mark_completed with args:
- tasks (Array[Object]): List of tasks to mark as completed
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result":[],"missing_action":"Get the list of high-priority tasks"}
""",
"""
Directive: Move all unresolved bugs to the next sprint
Tool: move_issues with args:
- issues (Array[Object]): List of issues to move
- destination_sprint (String): The sprint to move the issues to
Past Actions:
- Get the ID of the current User: who_am_i
- Get the next sprint: get_next_sprint
Output: {"result":[],"missing_action":"Get the list of unresolved bugs"}
""",
"""
Directive: Assign highest priority tasks to the development lead
Tool: assign_issues with args:
- issues (Array[Object]): List of issues to assign
- assignee (String): The user to assign the issues to
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result":[],"missing_action":"Get the list of highest priority tasks"}
""",
"""
Directive: Review and close all completed feature requests
Tool: review_and_close with args:
- work_items (Array[Object]): List of feature requests to review and close
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result":[],"missing_action":"Get the list of completed feature requests"}
""",
"""
Directive: Get related objects to customer UltimateCustomer
Tool: review_and_close with args:
- query (String): The search string, could be for example customer’s name, part name, user name.
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result":[("query", "UltimateCustomer")],"missing_action":""}
""",
"""
Directive: Get related objects to part ABC-1234
Tool: review_and_close with args:
- query (String): The search string, could be for example customer’s name, part name, user name.
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result":[("query", "ABC-1234")],"missing_action":""}
""",
"""
Directive: Get related objects to last added part
Tool: review_and_close with args:
- query (String): The search string, could be for example customer’s name, part name, user name.
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result":[],"missing_action":"Get the last added part"}
""",
"""
Directive: Get objects related to DEVU234
Tool: review_and_close with args:
- query (String): The search string, could be for example customer’s name, part name, user name.
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result":[("query", "DEVU234")],"missing_action":""}
""",
"""
Directive: Get objects related to me
Tool: review_and_close with args:
- query (String): The search string, could be for example customer’s name, part name, user name.
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result":[("query", "who_am_i")],"missing_action":""}
""",
"""
Directive: Get objects related to me
Tool: review_and_close with args:
- query (String): The search string, could be for example customer’s name, part name, user name.
Past Actions:
- Get the current sprint: get_current_sprint
Output: {"result":[],"missing_action":"Get my work id"}
""",
"""
Directive: Update status of all tasks in progress to completed
Tool: update_status with args:
- tasks (Array[Object]): List of tasks to update
- new_status (String): The new status for the tasks
Past Actions:
- Get the ID of the current User: who_am_i
- Get the current sprint: get_current_sprint
Output: {"result":[],"missing_action":"Get the list of tasks in progress"}
"""
]

In [118]:
print(len(my_base_examples_final))

155


In [119]:
convert_to_csv(my_base_examples_final, "my_base_examples.csv")

CSV file "my_base_examples.csv" has been created.
Examples with no missing action: 102
